In [2]:
# Washington Multiple AGN (WMAGN) Catalog DR1 - SMBH Binary SDSS/Optical Sources
# Author: R. W. Pfeifle
# Original Date Created: 10 Sept. 2020
# Last Revision Date: 2 Oct. 2020

# New Form Creation Date: 31 January 2022
# Last Revision: 1 June 2023

# Purpose: Combine various catalogs of double-peaked optically selected dual AGN candidates
# 

In [2]:
# Load in packages for pandas, astropy, etc. 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table
from astropy.io import ascii
from astropy.table import Column, MaskedColumn
from astropy.io.ascii import masked
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.cosmology import LambdaCDM 
from astroquery.simbad import Simbad
from astroquery.sdss import SDSS

cosmo = LambdaCDM(H0=70, Om0=0.3, Ode0=0.7) #Creating our choice of cosmology here...

pd.set_option('display.max_columns', 300) # Setting max number of rows per df to be the size of the df
pd.set_option('display.max_rows', None)

In [2]:
def name_to_coords(df,dfcol):
    if (len(dfcol[0])) == 14:
        df['Coordinates'] = dfcol.str.slice(start=1) # Stripping the J
        df['RA_test'] = df['Coordinates'].str.slice(start=0, stop=6) # Stripping the DEC parts 
        df['Dec_test'] = df['Coordinates'].str.slice(start=6, stop=13) # Stripping the RA parts
        df['RA'] = df['RA_test'].str.slice(start=0, stop=2)+":"+df['RA_test'].str.slice(start=2, stop=4)+":"+df['RA_test'].str.slice(start=4, stop=6) # Putting together the RA coordinates separated by colons
        df['Dec'] = df['Dec_test'].str.slice(start=0, stop=3)+":"+df['Dec_test'].str.slice(start=3, stop=5)+":"+df['Dec_test'].str.slice(start=5, stop=8) # Putting together the Dec coodinates separated by colons
        df.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)
        return
    #print(dfcol.apply(len))
    elif (len(dfcol[0])) == 18:
        df['Coordinates'] = dfcol#.str.slice(start=1) # Stripping the J
        df['RA_test'] = df['Coordinates'].str.slice(start=0, stop=9) # Stripping the DEC parts 
        df['Dec_test'] = df['Coordinates'].str.slice(start=9, stop=19) # Stripping the RA parts
        df['RA'] = df['RA_test'].str.slice(start=0, stop=2)+":"+df['RA_test'].str.slice(start=2, stop=4)+":"+df['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
        df['Dec'] = df['Dec_test'].str.slice(start=0, stop=3)+":"+df['Dec_test'].str.slice(start=3, stop=5)+":"+df['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
        df.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)
        return
    elif (len(dfcol[0])) == 19:
        df['Coordinates'] = dfcol.str.slice(start=1) # Stripping the J
        df['RA_test'] = df['Coordinates'].str.slice(start=0, stop=9) # Stripping the DEC parts 
        df['Dec_test'] = df['Coordinates'].str.slice(start=9, stop=19) # Stripping the RA parts
        df['RA'] = df['RA_test'].str.slice(start=0, stop=2)+":"+df['RA_test'].str.slice(start=2, stop=4)+":"+df['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
        df['Dec'] = df['Dec_test'].str.slice(start=0, stop=3)+":"+df['Dec_test'].str.slice(start=3, stop=5)+":"+df['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
        df.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)
        return
    #elif dfcol.apply(len) ==
    else:
        print('Error Encountered')
        

In [31]:
# Tsalmantza+2011 report the detection of 9 SMBH binary candidates
# Methodology: They search for systems exhibiting two sets of emission lines in the optical (using SDSS):
#    one broad and one narrow set of emission lines which are kinematically offset. A small separation would be 
#    expected in the case of Keplerian roation due to a binary. Here the narrow lines trace the galaxy, the broad
#    lines trace the active BH.
# Probes redshift 0.1-1.5, and applied fitting technique to SDSS QSO spectra as well as objects classified as galaxies
# All objects selected based on interesting features are at z<0.8
# There seems to be some overlap with Shen+2010 ****

# Loading in objects examined by Tsalmantza
tsalmantza = pd.read_csv('Tsalmantza2011/table1.csv', sep=',') #((Table.read('Tsalmantza2011/table1.dat', format='ascii')).to_pandas())#.drop(columns=['---'])

# Here, we're keeping only objects flagged as candidate binaries
tsalmantza = tsalmantza[(tsalmantza['Class']=='B') | (tsalmantza['Class']=='B;D') | (tsalmantza['Class']=='E;B') | (tsalmantza['Class']=='A;B') | (tsalmantza['Class']=='D;B') | (tsalmantza['Class']=='O;B')].reset_index(drop=True)

tsalmantza['Name1'] = tsalmantza['Name']
tsalmantza['Name2'] = tsalmantza['Name']
tsalmantza['z1'] = tsalmantza['z_NL']
tsalmantza['z2'] = tsalmantza['z_NL']
tsalmantza['z1_type'] = "spec"
tsalmantza['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(tsalmantza,tsalmantza['Name'])

# Converting the coordinates
coordconvert = SkyCoord(ra = tsalmantza['RA'], dec = tsalmantza['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
tsalmantza['RA_deg'] = coordconvert.ra.degree
tsalmantza['Dec_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
tsalmantza['RA2'] = tsalmantza['RA']
tsalmantza['Dec2'] = tsalmantza['Dec']

tsalmantza['RA2_deg'] = tsalmantza['RA_deg']
tsalmantza['Dec2_deg'] = tsalmantza['Dec_deg']

# Adding details about the coordinates
tsalmantza['Equinox'] = "J2000"
tsalmantza['Coordinate_waveband'] = "Optical"
tsalmantza['Coordinate_Source'] = "SDSS"

tsalmantza['System Type'] = 'Binary AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
tsalmantza['Brightness1'] = -100
tsalmantza['Brightness_band1'] = -100
tsalmantza['Brightness_type1'] = -100

tsalmantza['Brightness2'] = -100
tsalmantza['Brightness_band2'] = -100
tsalmantza['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
tsalmantza['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit



#tsalmantza['Sep(kpc)'] = tsalmantza['Sep']*((cosmo.arcsec_per_kpc_proper(tsalmantza['z']))**(-1))



# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
tsalmantza['delta_z'] = tsalmantza['z1']-tsalmantza['z2']
tsalmantza['dV'] = (2.99e+5)*((1+tsalmantza['z1'])**2 - (1+tsalmantza['z2'])**2)/((1+tsalmantza['z1'])**2+(1+tsalmantza['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
tsalmantza['Selection Method'] = "Double-Peaked Optical Spectroscopic Emission Lines" #DPSELs
tsalmantza['Confirmation Method'] = "-99"
tsalmantza['Paper(s)'] = "Tsalmantza+2011"
tsalmantza['BibCode(s)'] = "2011ApJ...738...20T"
tsalmantza['DOI(s)'] = "https://doi:10.1088/0004-637X/738/1/20"

## And dropping any columns that we don't need....
#tsalmantza.drop(labels=['SDSS','f_SDSS','Vel','logL','Type','Q','zr','zh'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#tsalmantza = tsalmantza[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#tsalmantza.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']

tsalmantza




# Eracleous+2012 report the detection of 
# Redshift range of z<0.7. 
# Trying to probe binaries with separations of subparsecs and periods of a few centuries
# They systematically search SDSS for quasars whose broad (single-peaked) B-beta lines are offset from the expected
# wavelength by 1000 km/s or more, considering these as candidate SMBH binaries or recoiling SMBHs
#     They throw out systems where delta v < 1000 km/s, and systems showing clear DPE signatures.
#     But they retain flat top line profiles as well as line profiles that are displayed but have an extended wing or shoulder
# ---> They stress this is not a complete sample, and is very subjective given that visual classification was used, 
#      and they added two objects from Strateva+03
# ----> There is overlap with Tsalmantza+ and others

# Bonning+200 attempted to find binaries or recoils ...



,Name,RA,Dec,MJD,Plate,Fiber,z_NL,z_BL,Method,Class,Name1,Name2,z1,z2,z1_type,z2_type,RA_deg,Dec_deg,RA2,Dec2,RA2_deg,Dec2_deg,Equinox,Coordinate_waveband,Coordinate_Source,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Sep,delta_z,dV,Selection Method,Confirmation Method,Paper(s),BibCode(s),DOI(s)
0,J0927+2943,09:27:12.65,+29:43:44.1,53389,1939,467,0.713,0.698,QG;QQ,B,J0927+2943,J0927+2943,0.713,0.713,spec,spec,141.802708,29.728917,09:27:12.65,+29:43:44.1,141.802708,29.728917,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Tsalmantza+2011,2011ApJ...738...20T,https://doi:10.1088/0004-637X/738/1/20
1,J0932+0318,09:32:01.60,+03:18:58.7,52254,568,39,0.420,0.401,QQ,B;D,J0932+0318,J0932+0318,0.420,0.420,spec,spec,143.006667,3.316306,09:32:01.60,+03:18:58.7,143.006667,3.316306,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Tsalmantza+2011,2011ApJ...738...20T,https://doi:10.1088/0004-637X/738/1/20
2,J1000+2233,10:00:21.80,+22:33:18.6,53737,2298,102,0.419,0.377,GQ,E;B,J1000+2233,J1000+2233,0.419,0.419,spec,spec,150.090833,22.555167,10:00:21.80,+22:33:18.6,150.090833,22.555167,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Tsalmantza+2011,2011ApJ...738...20T,https://doi:10.1088/0004-637X/738/1/20
3,J1012+2613,10:12:26.86,+26:13:27.3,53757,2347,513,0.378,0.351,QQ,E;B,J1012+2613,J1012+2613,0.378,0.378,spec,spec,153.111917,26.224250,10:12:26.86,+26:13:27.3,153.111917,26.224250,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Tsalmantza+2011,2011ApJ...738...20T,https://doi:10.1088/0004-637X/738/1/20
4,J1050+3456,10:50:41.36,+34:56:31.4,53431,2025,603,0.272,0.258,QQ;GQ,B,J1050+3456,J1050+3456,0.272,0.272,spec,spec,162.672333,34.942056,10:50:41.36,+34:56:31.4,162.672333,34.942056,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Tsalmantza+2011,2011ApJ...738...20T,https://doi:10.1088/0004-637X/738/1/20
5,J1154+0134,11:54:49.42,+01:34:43.6,52051,515,99,0.469,0.450,QQ,A;B,J1154+0134,J1154+0134,0.469,0.469,spec,spec,178.705917,1.578778,11:54:49.42,+01:34:43.6,178.705917,1.578778,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Tsalmantza+2011,2011ApJ...738...20T,https://doi:10.1088/0004-637X/738/1/20
6,J1536+0441,15:36:36.22,+04:41:27.0,54567,1836,270,0.389,0.373,QG;QQ,D;B,J1536+0441,J1536+0441,0.389,0.389,spec,spec,234.150917,4.690833,15:36:36.22,+04:41:27.0,234.150917,4.690833,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Tsalmantza+2011,2011ApJ...738...20T,https://doi:10.1088/0004-637X/738/1/20
7,J1539+3333,15:39:08.09,+33:33:28.0,52823,1355,572,0.226,0.199,QG;QQ,O;B,J1539+3333,J1539+3333,0.226,0.226,spec,spec,234.783708,33.557778,15:39:08.09,+33:33:28.0,234.783708,33.557778,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Tsalmantza+2011,2011ApJ...738...20T,https://doi:10.1088/0004-637X/738/1/20
8,J1714+3327,17:14:48.51,+33:27:38.3,54591,2973,190,0.181,0.186,QQ,O;B,J1714+3327,J1714+3327,0.181,0.181,spec,spec,258.702125,33.460639,17:14:48.51,+33:27:38.3,258.702125,33.460639,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Tsalmantza+2011,2011ApJ...738...20T,https://doi:10.1088/0004-637X/738/1/20


In [5]:
# Here we're loading in the Decarli+2013 and Lusso+ tables, which we'll need to match on the decarli2013+2011 table since \
# the sample in Decarli+2013 are pulled directly from the binary candidates in decarli2013+2011.

#Summary of the follow-up observations. (1) Quasar name. (2) and (3) Right ascension and declination (J2000). \
# (4) Object classification, based on the shape of Balmer lines: B – BHB candidates; D – DPEs; A – sources \
# with Asymmetric line profiles; O – Others. (5) Redshift of narrow lines. (6) Date of spectroscopic \
# follow-up observations. (7) Time lag between SDSS observations and our follow-up spectroscopy, in the \
# rest frame of zNL. (8) Telescope: C2.2m = Calar Alto 2.2 m; C3.5m = Calar Alto 3.5 m; Lick; Keck. (9) Weather \
# conditions during the spectroscopic follow-up: P = photometric; NP = not photometric; C = cloudy.

decarli2013 = pd.read_csv('Tables/Decarli2013/Decarli2013.csv', sep=',')

#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# Here, we're keeping only objects flagged as candidate binaries
#decarli2013 = decarli2013[(decarli2013['Class']=='B') | (decarli2013['Class']=='B;D') | (decarli2013['Class']=='E;B') | (decarli2013['Class']=='A;B') | (decarli2013['Class']=='D;B') | (decarli2013['Class']=='O;B')].reset_index(drop=True)

decarli2013['Name1'] = decarli2013['Objname']
decarli2013['Name2'] = decarli2013['Objname']
decarli2013['z1'] = decarli2013['zNL']
decarli2013['z2'] = decarli2013['zNL']
decarli2013['z1_type'] = "spec"
decarli2013['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(decarli2013,decarli2013['Name'])

# Converting the coordinates
coordconvert = SkyCoord(ra = decarli2013['RA'], dec = decarli2013['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
decarli2013['RA1_deg'] = coordconvert.ra.degree
decarli2013['Dec1_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
decarli2013['RA2'] = decarli2013['RA']
decarli2013['Dec2'] = decarli2013['Dec']

decarli2013['RA2_deg'] = decarli2013['RA1_deg']
decarli2013['Dec2_deg'] = decarli2013['Dec1_deg']

# Adding details about the coordinates
decarli2013['Equinox'] = "J2000"
decarli2013['Coordinate_waveband'] = "Optical"
decarli2013['Coordinate_Source'] = "SDSS"

decarli2013['System Type'] = 'Binary AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
decarli2013['Brightness1'] = -100
decarli2013['Brightness_band1'] = -100
decarli2013['Brightness_type1'] = -100

decarli2013['Brightness2'] = -100
decarli2013['Brightness_band2'] = -100
decarli2013['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
decarli2013['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit



#decarli2013['Sep(kpc)'] = decarli2013['Sep']*((cosmo.arcsec_per_kpc_proper(decarli2013['z']))**(-1))



# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
decarli2013['delta_z'] = decarli2013['z1']-decarli2013['z2']
decarli2013['dV'] = (2.99e+5)*((1+decarli2013['z1'])**2 - (1+decarli2013['z2'])**2)/((1+decarli2013['z1'])**2+(1+decarli2013['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
decarli2013['Selection Method'] = "Velocity Shifts between Narrow and Broad Lines" #DPSELs
decarli2013['Confirmation Method'] = "-99"
decarli2013['Paper(s)'] = "Decarli+2013"
decarli2013['BibCode(s)'] = "2013MNRAS.433.1492D"
decarli2013['DOI(s)'] = "https://doi.org/10.1093/mnras/stt831"
decarli2013['Notes'] = ' '
## And dropping any columns that we don't need....
#decarli2013.drop(labels=['SDSS','f_SDSS','Vel','logL','Type','Q','zr','zh'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#decarli2013 = decarli2013[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#decarli2013.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']


decarli2013


,Objname,RA,Dec,Class,zNL,Date,delta_t0,Tel,Weather,Name1,Name2,z1,z2,z1_type,z2_type,RA1_deg,Dec1_deg,RA2,Dec2,RA2_deg,Dec2_deg,Equinox,Coordinate_waveband,Coordinate_Source,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Sep,delta_z,dV,Selection Method,Confirmation Method,Paper(s),BibCode(s),DOI(s),Notes
0,J0012-1022,00:12:24.03,-10:22:26.3,A,0.228,2011-07-22,8.08,C3.5m,P,J0012-1022,J0012-1022,0.228,0.228,spec,spec,3.100125,-10.373972,00:12:24.03,-10:22:26.3,3.100125,-10.373972,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Velocity Shifts between Narrow and Broad Lines,-99,Decarli+2013,2013MNRAS.433.1492D,https://doi.org/10.1093/mnras/stt831,
1,J0155-0857,01:55:30.02,-08:57:04.0,O,0.165,2011-09-26,8.62,Lick,C,J0155-0857,J0155-0857,0.165,0.165,spec,spec,28.875083,-8.951111,01:55:30.02,-08:57:04.0,28.875083,-8.951111,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Velocity Shifts between Narrow and Broad Lines,-99,Decarli+2013,2013MNRAS.433.1492D,https://doi.org/10.1093/mnras/stt831,
2,J0221+0101,02:21:13.15,+01:01:02.9,O,0.354,2011-09-26,8.02,Lick,C,J0221+0101,J0221+0101,0.354,0.354,spec,spec,35.304792,1.017472,02:21:13.15,+01:01:02.9,35.304792,1.017472,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Velocity Shifts between Narrow and Broad Lines,-99,Decarli+2013,2013MNRAS.433.1492D,https://doi.org/10.1093/mnras/stt831,
3,J0829+2728,08:29:30.60,+27:28:22.7,O,0.321,2012-02-22,6.32,C2.2m,NP,J0829+2728,J0829+2728,0.321,0.321,spec,spec,127.377500,27.472972,08:29:30.60,+27:28:22.7,127.377500,27.472972,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Velocity Shifts between Narrow and Broad Lines,-99,Decarli+2013,2013MNRAS.433.1492D,https://doi.org/10.1093/mnras/stt831,
4,J0918+3156,09:18:33.82,+31:56:21.2,O,0.452,2012-02-20,5.64,C2.2m,NP,J0918+3156,J0918+3156,0.452,0.452,spec,spec,139.640917,31.939222,09:18:33.82,+31:56:21.2,139.640917,31.939222,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Velocity Shifts between Narrow and Broad Lines,-99,Decarli+2013,2013MNRAS.433.1492D,https://doi.org/10.1093/mnras/stt831,
5,J0919+1108,09:19:30.32,+11:08:54.0,O,0.369,2012-02-20,5.86,C2.2m,NP,J0919+1108,J0919+1108,0.369,0.369,spec,spec,139.876333,11.148333,09:19:30.32,+11:08:54.0,139.876333,11.148333,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Velocity Shifts between Narrow and Broad Lines,-99,Decarli+2013,2013MNRAS.433.1492D,https://doi.org/10.1093/mnras/stt831,
6,J0921+3835,09:21:16.13,+38:35:37.6,A,0.187,2012-02-22,7.50,C2.2m,NP,J0921+3835,J0921+3835,0.187,0.187,spec,spec,140.317208,38.593778,09:21:16.13,+38:35:37.6,140.317208,38.593778,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Velocity Shifts between Narrow and Broad Lines,-99,Decarli+2013,2013MNRAS.433.1492D,https://doi.org/10.1093/mnras/stt831,
7,J0927+2943,09:27:12.65,+29:43:44.1,B,0.713,2012-02-20,4.14,C2.2m,NP,J0927+2943,J0927+2943,0.713,0.713,spec,spec,141.802708,29.728917,09:27:12.65,+29:43:44.1,141.802708,29.728917,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Velocity Shifts between Narrow and Broad Lines,-99,Decarli+2013,2013MNRAS.433.1492D,https://doi.org/10.1093/mnras/stt831,
8,J0931+3204,09:31:39.05,+32:04:00.2,O,0.226,2012-02-21,5.79,C2.2m,NP,J0931+3204,J0931+3204,0.226,0.226,spec,spec,142.912708,32.066722,09:31:39.05,+32:04:00.2,142.912708,32.066722,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Velocity Shifts between Narrow and Broad Lines,-99,Decarli+2013,2013MNRAS.433.1492D,https://doi.org/10.1093/mnras/stt831,
9,J0932+0318,09:32:01.60,+03:18:58.7,B;D,0.420,2012-02-22,7.19,C2.2m,NP,J0932+0318,J0932+0318,0.420,0.420,spec,spec,143.006667,3.316306,09:32:01.60,+03:18:58.7,143.006667,3.316306,J2000,Optical,SDSS,Binary AGN C

In [6]:
# Here we'll add Lusso+2014, which was a follow-up to Tsalmantza+2011 and Decarli+2013

#for index, row in decarli2013.iterrows():
    #if row['Name'] in hutsemekers2020:
decarli2013['Paper(s)'] += ' ; Lusso+2014 '
decarli2013['BibCode(s)'] += ' ; 2014MNRAS.441..316L' 
decarli2013['DOI(s)'] += ' ;  https://doi.org/10.1093/mnras/stu572'
decarli2013['Notes']+=' Lusso+ fit the SEDs to search for evidence of peculiar properties that could be consistent with the recoil hypothesis. In a scenario where teh recoilis displaced from the center of the galaxy and has left the torus behind the AGN should show a deficit of 12um emission. They find that all objects in the sample (with the possible exception of J1154) exhibit a significant amount of 12um emission in contrast to the recoil scenario.'

decarli2013

,Objname,RA,Dec,Class,zNL,Date,delta_t0,Tel,Weather,Name1,Name2,z1,z2,z1_type,z2_type,RA1_deg,Dec1_deg,RA2,Dec2,RA2_deg,Dec2_deg,Equinox,Coordinate_waveband,Coordinate_Source,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Sep,delta_z,dV,Selection Method,Confirmation Method,Paper(s),BibCode(s),DOI(s),Notes
0,J0012-1022,00:12:24.03,-10:22:26.3,A,0.228,2011-07-22,8.08,C3.5m,P,J0012-1022,J0012-1022,0.228,0.228,spec,spec,3.100125,-10.373972,00:12:24.03,-10:22:26.3,3.100125,-10.373972,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Velocity Shifts between Narrow and Broad Lines,-99,Decarli+2013 ; Lusso+2014,2013MNRAS.433.1492D ; 2014MNRAS.441..316L,https://doi.org/10.1093/mnras/stt831 ; https:...,Lusso+ fit the SEDs to search for evidence o...
1,J0155-0857,01:55:30.02,-08:57:04.0,O,0.165,2011-09-26,8.62,Lick,C,J0155-0857,J0155-0857,0.165,0.165,spec,spec,28.875083,-8.951111,01:55:30.02,-08:57:04.0,28.875083,-8.951111,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Velocity Shifts between Narrow and Broad Lines,-99,Decarli+2013 ; Lusso+2014,2013MNRAS.433.1492D ; 2014MNRAS.441..316L,https://doi.org/10.1093/mnras/stt831 ; https:...,Lusso+ fit the SEDs to search for evidence o...
2,J0221+0101,02:21:13.15,+01:01:02.9,O,0.354,2011-09-26,8.02,Lick,C,J0221+0101,J0221+0101,0.354,0.354,spec,spec,35.304792,1.017472,02:21:13.15,+01:01:02.9,35.304792,1.017472,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Velocity Shifts between Narrow and Broad Lines,-99,Decarli+2013 ; Lusso+2014,2013MNRAS.433.1492D ; 2014MNRAS.441..316L,https://doi.org/10.1093/mnras/stt831 ; https:...,Lusso+ fit the SEDs to search for evidence o...
3,J0829+2728,08:29:30.60,+27:28:22.7,O,0.321,2012-02-22,6.32,C2.2m,NP,J0829+2728,J0829+2728,0.321,0.321,spec,spec,127.377500,27.472972,08:29:30.60,+27:28:22.7,127.377500,27.472972,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Velocity Shifts between Narrow and Broad Lines,-99,Decarli+2013 ; Lusso+2014,2013MNRAS.433.1492D ; 2014MNRAS.441..316L,https://doi.org/10.1093/mnras/stt831 ; https:...,Lusso+ fit the SEDs to search for evidence o...
4,J0918+3156,09:18:33.82,+31:56:21.2,O,0.452,2012-02-20,5.64,C2.2m,NP,J0918+3156,J0918+3156,0.452,0.452,spec,spec,139.640917,31.939222,09:18:33.82,+31:56:21.2,139.640917,31.939222,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Velocity Shifts between Narrow and Broad Lines,-99,Decarli+2013 ; Lusso+2014,2013MNRAS.433.1492D ; 2014MNRAS.441..316L,https://doi.org/10.1093/mnras/stt831 ; https:...,Lusso+ fit the SEDs to search for evidence o...
5,J0919+1108,09:19:30.32,+11:08:54.0,O,0.369,2012-02-20,5.86,C2.2m,NP,J0919+1108,J0919+1108,0.369,0.369,spec,spec,139.876333,11.148333,09:19:30.32,+11:08:54.0,139.876333,11.148333,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Velocity Shifts between Narrow and Broad Lines,-99,Decarli+2013 ; Lusso+2014,2013MNRAS.433.1492D ; 2014MNRAS.441..316L,https://doi.org/10.1093/mnras/stt831 ; https:...,Lusso+ fit the SEDs to search for evidence o...
6,J0921+3835,09:21:16.13,+38:35:37.6,A,0.187,2012-02-22,7.50,C2.2m,NP,J0921+3835,J0921+3835,0.187,0.187,spec,spec,140.317208,38.593778,09:21:16.13,+38:35:37.6,140.317208,38.593778,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Velocity Shifts between Narrow and Broad Lines,-99,Decarli+2013 ; Lusso+2014,2013MNRAS.433.1492D ; 2014MNRAS.441..316L,https://doi.org/10.1093/mnras/stt831 ; https:...,Lusso+ fit the SEDs to search for evidence o...
7,J0927+2943,09:27:12.65,+29:43:44.1,B,0.713,2012-02-20,4.14,C2.2m,NP,J0927+2943,J0927+2943,0.713,0.713,spec,spec,141.802708,29.728917,09:27:12.65,+29:43:44.1,141.802708,29.728917,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Velocity Shifts between Narrow and Broad Lines,-99,Decarl

,Name,RA,Dec,MJD,Plate,Fiber,z_NL,z_BL,Method,Class
0,J0927+2943,09:27:12.65,+29:43:44.1,53389,1939,467,0.713,0.698,QG;QQ,B
1,J0932+0318,09:32:01.60,+03:18:58.7,52254,568,39,0.420,0.401,QQ,B;D
2,J1000+2233,10:00:21.80,+22:33:18.6,53737,2298,102,0.419,0.377,GQ,E;B
3,J1012+2613,10:12:26.86,+26:13:27.3,53757,2347,513,0.378,0.351,QQ,E;B
4,J1050+3456,10:50:41.36,+34:56:31.4,53431,2025,603,0.272,0.258,QQ;GQ,B
5,J1154+0134,11:54:49.42,+01:34:43.6,52051,515,99,0.469,0.450,QQ,A;B
6,J1536+0441,15:36:36.22,+04:41:27.0,54567,1836,270,0.389,0.373,QG;QQ,D;B
7,J1539+3333,15:39:08.09,+33:33:28.0,52823,1355,572,0.226,0.199,QG;QQ,O;B
8,J1714+3327,17:14:48.51,+33:27:38.3,54591,2973,190,0.181,0.186,QQ,O;B


In [28]:
# Here we're loading in the Shen+2013 catalog and we are ONLY including objects with their binary candidate flag \
# indicated by the (1) flag in  the 'Category' column
# I am not including ambiguous (3) or variability (2) cases
shen2013 = pd.read_csv('Tables/Shen2013/table2.csv', sep=',')
shen2013 = shen2013[shen2013['Category']==1.0]

shen2013['Name1'] = shen2013['Designation']
shen2013['Name2'] = shen2013['Designation']
shen2013['z1'] = shen2013['z_sys']
shen2013['z2'] = shen2013['z_sys']
shen2013['z1_type'] = "spec"
shen2013['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(shen2013,shen2013['Designation'])

shen2013['Coordinates'] = shen2013['Designation']#.str.slice(start=1) # Stripping the J
shen2013['RA_test'] = shen2013['Coordinates'].str.slice(start=0, stop=9) # Stripping the DEC parts 
shen2013['Dec_test'] = shen2013['Coordinates'].str.slice(start=9, stop=19) # Stripping the RA parts
shen2013['RA'] = shen2013['RA_test'].str.slice(start=0, stop=2)+":"+shen2013['RA_test'].str.slice(start=2, stop=4)+":"+shen2013['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
shen2013['Dec'] = shen2013['Dec_test'].str.slice(start=0, stop=3)+":"+shen2013['Dec_test'].str.slice(start=3, stop=5)+":"+shen2013['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
shen2013.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)

# Converting the coordinates
coordconvert = SkyCoord(ra = shen2013['RA'], dec = shen2013['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
shen2013['RA_deg'] = coordconvert.ra.degree
shen2013['Dec_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
shen2013['RA2'] = shen2013['RA']
shen2013['Dec2'] = shen2013['Dec']

shen2013['RA2_deg'] = shen2013['RA_deg']
shen2013['Dec2_deg'] = shen2013['Dec_deg']

# Adding details about the coordinates
shen2013['Equinox'] = "J2000"
shen2013['Coordinate_waveband'] = "Optical"
shen2013['Coordinate_Source'] = "SDSS"

shen2013['System Type'] = 'Binary AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
shen2013['Brightness1'] = -100
shen2013['Brightness_band1'] = -100
shen2013['Brightness_type1'] = -100

shen2013['Brightness2'] = -100
shen2013['Brightness_band2'] = -100
shen2013['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
shen2013['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit



#shen2013['Sep(kpc)'] = shen2013['Sep']*((cosmo.arcsec_per_kpc_proper(shen2013['z']))**(-1))



# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
shen2013['delta_z'] = shen2013['z1']-shen2013['z2']
shen2013['dV'] = (2.99e+5)*((1+shen2013['z1'])**2 - (1+shen2013['z2'])**2)/((1+shen2013['z1'])**2+(1+shen2013['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
shen2013['Selection Method'] = "Extreme Velocity Offsets / Radial Velocity Shifts" #DPSELs
shen2013['Confirmation Method'] = "-99"
shen2013['Paper(s)'] = "Shen+2013"
shen2013['BibCode(s)'] = "2013ApJ...775...49S"
shen2013['DOI(s)'] = "https://doi:10.1088/0004-637X/775/1/49"

## And dropping any columns that we don't need....
#shen2013.drop(labels=['SDSS','f_SDSS','Vel','logL','Type','Q','zr','zh'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#shen2013 = shen2013[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#shen2013.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']
#


shen2013

,Designation,z_sys,logM_1(M_sun),R_BLR(pc),E1Plate,E1Fiber,E1MJD,E2Plate,E2Fiber,E2MJD,Deltat(days),$V_{\rm ccf}^*$(km s^-1),$V_{\rm ccf}^*$(km s^-1).1,$V_{\rm ccf}^*$(km s^-1).2,$a_{\rm ccf}^*$(km s^-1 yr^-1),$a_{\rm ccf}^*$(km s^-1 yr^-1).1,$a_{\rm ccf}^*$(km s^-1 yr^-1).2,Category,Name1,Name2,z1,z2,z1_type,z2_type,RA,Dec,RA_deg,Dec_deg,RA2,Dec2,RA2_deg,Dec2_deg,Equinox,Coordinate_waveband,Coordinate_Source,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Sep,delta_z,dV,Selection Method,Confirmation Method,Paper(s),BibCode(s),DOI(s)
3,030100.23+000429.3,0.486,8.50,0.056,411,265,51817,709,477,52205,261,420,163,545,588,229.0,762.0,1.0,030100.23+000429.3,030100.23+000429.3,0.486,0.486,spec,spec,03:01:00.23,+00:04:29.3,45.250958,0.074806,03:01:00.23,+00:04:29.3,45.250958,0.074806,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Extreme Velocity Offsets / Radial Velocity Shifts,-99,Shen+2013,2013ApJ...775...49S,https://doi:10.1088/0004-637X/775/1/49
7,032213.89+005513.4,0.185,8.04,0.068,414,341,51901,1181,329,53358,1229,48,21,78,14,6.0,23.0,1.0,032213.89+005513.4,032213.89+005513.4,0.185,0.185,spec,spec,03:22:13.89,+00:55:13.4,50.557875,0.920389,03:22:13.89,+00:55:13.4,50.557875,0.920389,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Extreme Velocity Offsets / Radial Velocity Shifts,-99,Shen+2013,2013ApJ...775...49S,https://doi:10.1088/0004-637X/775/1/49
13,122909.52-003530.0,0.450,8.84,0.056,289,150,51990,2568,32,54153,1491,324,197,453,79,48.0,111.0,1.0,122909.52-003530.0,122909.52-003530.0,0.450,0.450,spec,spec,12:29:09.52,-00:35:30.0,187.289667,-0.591667,12:29:09.52,-00:35:30.0,187.289667,-0.591667,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Extreme Velocity Offsets / Radial Velocity Shifts,-99,Shen+2013,2013ApJ...775...49S,https://doi:10.1088/0004-637X/775/1/49
18,141020.57+364322.7,0.450,8.44,0.053,1643,359,53143,2931,496,54590,998,227,142,309,83,52.0,113.0,1.0,141020.57+364322.7,141020.57+364322.7,0.450,0.450,spec,spec,14:10:20.57,+36:43:22.7,212.585708,36.722972,14:10:20.57,+36:43:22.7,212.585708,36.722972,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Extreme Velocity Offsets / Radial Velocity Shifts,-99,Shen+2013,2013ApJ...775...49S,https://doi:10.1088/0004-637X/775/1/49
19,153705.95+005522.8,0.137,7.63,0.038,2955,289,54562,315,419,51663,2550,110,43,181,15,6.0,26.0,1.0,153705.95+005522.8,153705.95+005522.8,0.137,0.137,spec,spec,15:37:05.95,+00:55:22.8,234.274792,0.923000,15:37:05.95,+00:55:22.8,234.274792,0.923000,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Extreme Velocity Offsets / Radial Velocity Shifts,-99,Shen+2013,2013ApJ...775...49S,https://doi:10.1088/0004-637X/775/1/49
21,155053.16+052112.1,0.110,8.96,0.039,1822,308,53172,2951,326,54592,1278,-1034,-1183,-882,-295,-338.0,-251.0,1.0,155053.16+052112.1,155053.16+052112.1,0.110,0.110,spec,spec,15:50:53.16,+05:21:12.1,237.721500,5.353361,15:50:53.16,+05:21:12.1,237.721500,5.353361,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Extreme Velocity Offsets / Radial Velocity Shifts,-99,Shen+2013,2013ApJ...775...49S,https://doi:10.1088/0004-637X/775/1/49
28,234932.77-003645.8,0.279,8.26,0.068,386,215,51788,684,261,52523,574,-68,-118,-17,-43,-75.0,-11.0,1.0,234932.77-003645.8,234932.77-003645.8,0.279,0.279,spec,spec,23:49:32.77,-00:36:45.8,357.386542,-0.612722,23:49:32.77,-00:36:45.8,357.386542,-0.612722,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Extreme Velocity Offsets / Radial Velocity Shifts,-99,Shen+2013,2013ApJ...775...49S,https://doi:10.1088/0004-637X/775/1/49


In [18]:
# Here we're loading in the Eracleous+2012 table(s)

erac2012 = ((Table.read('Eracleous2012/table1.dat', readme = 'Eracleous2012/ReadMe', format='ascii.cds')).to_pandas())#.drop(columns=['---'])

# Need to add in other info for the 68 candiates follow-up and the 14 specific candidates they discuss



erac2012.drop_duplicates(subset=['SDSS'], inplace=True)
erac2012.reset_index(drop=True, inplace=True)

erac2012['Name1'] = erac2012['SDSS']
erac2012['Name2'] = erac2012['SDSS']
erac2012['z1'] = erac2012['z']
erac2012['z2'] = erac2012['z']
erac2012['z1_type'] = "spec"
erac2012['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
name_to_coords(erac2012,erac2012['SDSS'])

# Converting the coordinates
coordconvert = SkyCoord(ra = erac2012['RA'], dec = erac2012['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
erac2012['RA_deg'] = coordconvert.ra.degree
erac2012['Dec_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
erac2012['RA2'] = erac2012['RA']
erac2012['Dec2'] = erac2012['Dec']

erac2012['RA2_deg'] = erac2012['RA_deg']
erac2012['Dec2_deg'] = erac2012['Dec_deg']

# Adding details about the coordinates
erac2012['Equinox'] = "J2000"
erac2012['Coordinate_waveband'] = "Optical"
erac2012['Coordinate_Source'] = "SDSS"

erac2012['System Type'] = 'Binary AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
erac2012['Brightness1'] = erac2012['Vmag']
erac2012['Brightness_band1'] = 'V'
erac2012['Brightness_type1'] = 'apparent mag'

erac2012['Brightness2'] = erac2012['Vmag']
erac2012['Brightness_band2'] = 'V'
erac2012['Brightness_type2'] = 'apparent mag'

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
erac2012['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit


#erac2012['Sep(kpc)'] = erac2012['Sep']*((cosmo.arcsec_per_kpc_proper(erac2012['z']))**(-1))


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
erac2012['delta_z'] = erac2012['z1']-erac2012['z2']
erac2012['dV'] = (2.99e+5)*((1+erac2012['z1'])**2 - (1+erac2012['z2'])**2)/((1+erac2012['z1'])**2+(1+erac2012['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
erac2012['Selection Method'] = "Double-Peaked Broad Optical Spectroscopic Emission Lines" #DPSELs
erac2012['Confirmation Method'] = "-99"
erac2012['Paper(s)'] = "Eracleous+2012 ; Runnoe+2015 ; Runnoe+2017 ; Nguyen+2020"
erac2012['BibCode(s)'] = "2012ApJS..201...23E ; 2015ApJS..221....7R ; 2017MNRAS.468.1683R ; 2020ApJ...894..105N"
erac2012['DOI(s)'] = "doi:10.1088/0067-0049/201/2/23 ; https://doi.org/10.1088/0067-0049/221/1/7 ; https://doi.org/10.1093/mnras/stx452 ; https://doi.org/10.3847/1538-4357/ab88b5"

## And dropping any columns that we don't need....
#erac2012.drop(labels=['SDSS','f_SDSS','Vel','logL','Type','Q','zr','zh'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#erac2012 = erac2012[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#erac2012.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']
#


# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# Note Runnoe+ and Nyugen+ may have some info we want to include here, but for now I'm just listing their paper
# info since they looked at all of the same 88 objects

erac2012

,SDSS,z,Vmag,AV,VMag,Date,Instr,Exp,S/N,lam1,lam2,Name1,Name2,z1,z2,z1_type,z2_type,RA,Dec,RA_deg,Dec_deg,RA2,Dec2,RA2_deg,Dec2_deg,Equinox,Coordinate_waveband,Coordinate_Source,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Sep,delta_z,dV,Selection Method,Confirmation Method,Paper(s),BibCode(s),DOI(s)
0,001224.02-102226.2,0.2287,17.07,0.127,-23.27,2001/08/20,S,3756,29,3093,7505,001224.02-102226.2,001224.02-102226.2,0.2287,0.2287,spec,spec,00:12:24.02,-10:22:26.2,3.100083,-10.373944,00:12:24.02,-10:22:26.2,3.100083,-10.373944,J2000,Optical,SDSS,Binary AGN Candidate,17.07,V,apparent mag,17.07,V,apparent mag,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Eracleous+2012,2012ApJS..201...23E,doi:10.1088/0067-0049/201/2/23
1,002444.10+003221.3,0.4024,16.85,0.083,-24.86,2000/12/22,S,9900,88,2709,6594,002444.10+003221.3,002444.10+003221.3,0.4024,0.4024,spec,spec,00:24:44.10,+00:32:21.3,6.183750,0.539250,00:24:44.10,+00:32:21.3,6.183750,0.539250,J2000,Optical,SDSS,Binary AGN Candidate,16.85,V,apparent mag,16.85,V,apparent mag,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Eracleous+2012,2012ApJS..201...23E,doi:10.1088/0067-0049/201/2/23
2,015530.01-085704.0,0.1648,16.84,0.080,-22.65,2001/09/16,S,2702,32,3263,7917,015530.01-085704.0,015530.01-085704.0,0.1648,0.1648,spec,spec,01:55:30.01,-08:57:04.0,28.875042,-8.951111,01:55:30.01,-08:57:04.0,28.875042,-8.951111,J2000,Optical,SDSS,Binary AGN Candidate,16.84,V,apparent mag,16.84,V,apparent mag,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Eracleous+2012,2012ApJS..201...23E,doi:10.1088/0067-0049/201/2/23
3,020011.53-093126.2,0.3602,17.96,0.081,-23.46,2001/08/28,S,8104,34,2795,6780,020011.53-093126.2,020011.53-093126.2,0.3602,0.3602,spec,spec,02:00:11.53,-09:31:26.2,30.048042,-9.523944,02:00:11.53,-09:31:26.2,30.048042,-9.523944,J2000,Optical,SDSS,Binary AGN Candidate,17.96,V,apparent mag,17.96,V,apparent mag,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Eracleous+2012,2012ApJS..201...23E,doi:10.1088/0067-0049/201/2/23
4,021259.59-003029.5,0.3945,17.87,0.117,-23.83,2000/09/29,S,2700,29,2714,6613,021259.59-003029.5,021259.59-003029.5,0.3945,0.3945,spec,spec,02:12:59.59,-00:30:29.5,33.248292,-0.508194,02:12:59.59,-00:30:29.5,33.248292,-0.508194,J2000,Optical,SDSS,Binary AGN Candidate,17.87,V,apparent mag,17.87,V,apparent mag,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Eracleous+2012,2012ApJS..201...23E,doi:10.1088/0067-0049/201/2/23
5,022014.57-072859.1,0.2137,18.65,0.078,-21.47,2001/09/10,S,4501,11,3133,7598,022014.57-072859.1,022014.57-072859.1,0.2137,0.2137,spec,spec,02:20:14.57,-07:28:59.1,35.060708,-7.483083,02:20:14.57,-07:28:59.1,35.060708,-7.483083,J2000,Optical,SDSS,Binary AGN Candidate,18.65,V,apparent mag,18.65,V,apparent mag,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Eracleous+2012,2012ApJS..201...23E,doi:10.1088/0067-0049/201/2/23
6,031715.87+425150.6,0.2356,19.03,0.616,-21.87,2003/12/03,S,13001,17,3075,7463,031715.87+425150.6,031715.87+425150.6,0.2356,0.2356,spec,spec,03:17:15.87,+42:51:50.6,49.316125,42.864056,03:17:15.87,+42:51:50.6,49.316125,42.864056,J2000,Optical,SDSS,Binary AGN Candidate,19.03,V,apparent mag,19.03,V,apparent mag,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Eracleous+2012,2012ApJS..201...23E,doi:10.1088/0067-0049/201/2/23
7,074007.28+410903.6,0.2347,17.78,0.161,-22.66,2000/12/06,S,5400,23,3074,7468,074007.28+410903.6,074007.28+410903.6,0.2347,0.2347,spec,spec,07:40:07.28,+41:09:03.6,115.030333,41.151000,07:40:07.28,+41:09:03.6,115.030333,41.151000,J2000,Optical,SDSS,Binary AGN Candidate,17.78,V,apparent mag,17.78,V,apparent mag,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Eracleous+2012,2012ApJS..201...23E,doi:10.1088/0067-0049/201/2/23
8,074157.25+275519.8,0.3252,18.52,0.170,-22.73,2002/03/06,S,4802,19,2867,6958,074157.25+275519.8,074157.25+275519.8,0.32

In [9]:
# Here we're loading in the Ju+2013 table(s)

ju2013 = pd.read_csv('Tables/Ju2013/Ju2013.csv', sep=',')
#(M_sun),(yr),(km s^-1),(km s^-1),(km s^-1),(pc),
#Notes. (1) Object name in the SDSS catalog. (2) Redshift of the object from the SDSS catalog \
#(Bolton et al. 2012). (3) Absolute g-band magnitude. (4) BH mass of the object measured from the \
# Mg II widths (Shen et al. 2011b). (5) The time interval between multiple observations. (6) The velocity shift \
# of Mg II measured by cross-correlation over the whole wavelength range [1700 Aring, 4000 Aring]. \
# (7) The velocity shift of Mg II measured by cross-correlation over the wavelength range near Mg II \
# [2700 Aring, 2900 Aring]. (8) The uncertainties of velocity shifts reported from peak hunting of \
# cross-correlation functions. (9) The upper limit for the separation of the SMBH binary assuming a mass \
# ratio of q = 1. The true separations are equal to these values only when the observer sits at the ideal \
# position for watching velocity drifts: an edge-on orbit with the secondary BH just intersecting the line \
# connecting the primary BH and the observer. (10) The date when the last SDSS observation of the quasar was \
# taken, in the format of MM-DD-YYYY. * The more reliable candidates that pass the test cross-correlation over \
# [1700 Aring, 3800 Aring] (see Section 4.1). m: for these quasars, one of the two epoch spectra has a \
# "marginal" plate quality flagged in SDSS platelist. These plates get the "marginal" flag because either \
# the minimum S/N of all the fibers is smaller than 15.0 or the number of exposures is smaller than 3. b: \
# one of the two epoch spectra has a "bad" plate quality flag. This flag reflects the very poor seeing \
# conditions and bright sky when the data were taken, and may lead to large spectrophotometric uncertainty. \
# However, for this candidate, most of the spurious features are introduced in the red end of the spectrum, \
# far from the Mg II line. b+m: one of the two epoch spectra has a bad plate quality flag, while the other has \
# a marginal flag.

ju2013['Name1'] = ju2013['SDSS']
ju2013['Name2'] = ju2013['SDSS']
ju2013['z1'] = ju2013['z']
ju2013['z2'] = ju2013['z']
ju2013['z1_type'] = "spec"
ju2013['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(ju2013,ju2013['Designation'])

ju2013['Coordinates'] = ju2013['SDSS']#.str.slice(start=1) # Stripping the J
ju2013['RA_test'] = ju2013['Coordinates'].str.slice(start=1, stop=10) # Stripping the DEC parts 
ju2013['Dec_test'] = ju2013['Coordinates'].str.slice(start=10, stop=19) # Stripping the RA parts
ju2013['RA'] = ju2013['RA_test'].str.slice(start=0, stop=2)+":"+ju2013['RA_test'].str.slice(start=2, stop=4)+":"+ju2013['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
ju2013['Dec'] = ju2013['Dec_test'].str.slice(start=0, stop=3)+":"+ju2013['Dec_test'].str.slice(start=3, stop=5)+":"+ju2013['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
ju2013.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)

# Converting the coordinates
coordconvert = SkyCoord(ra = ju2013['RA'], dec = ju2013['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
ju2013['RA_deg'] = coordconvert.ra.degree
ju2013['Dec_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
ju2013['RA2'] = ju2013['RA']
ju2013['Dec2'] = ju2013['Dec']

ju2013['RA2_deg'] = ju2013['RA_deg']
ju2013['Dec2_deg'] = ju2013['Dec_deg']

# Adding details about the coordinates
ju2013['Equinox'] = "J2000"
ju2013['Coordinate_waveband'] = "Optical"
ju2013['Coordinate_Source'] = "SDSS"

ju2013['System Type'] = 'Binary AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
ju2013['Brightness1'] = -100
ju2013['Brightness_band1'] = -100
ju2013['Brightness_type1'] = -100

ju2013['Brightness2'] = -100
ju2013['Brightness_band2'] = -100
ju2013['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
ju2013['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit


#ju2013['Sep(kpc)'] = ju2013['Sep']*((cosmo.arcsec_per_kpc_proper(ju2013['z']))**(-1))


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
ju2013['delta_z'] = ju2013['z1']-ju2013['z2']
ju2013['dV'] = (2.99e+5)*((1+ju2013['z1'])**2 - (1+ju2013['z2'])**2)/((1+ju2013['z1'])**2+(1+ju2013['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
ju2013['Selection Method'] = "Double-Peaked Optical Spectroscopic Emission Lines" #DPSELs
ju2013['Confirmation Method'] = "-99"
ju2013['Paper(s)'] = "Ju+2013"
ju2013['BibCode(s)'] = "2013ApJ...777...44J"
ju2013['DOI(s)'] = "https://doi.org/10.1088/0004-637X/777/1/44"
ju2013['Notes'] = " "


## And dropping any columns that we don't need....
#ju2013.drop(labels=['SDSS','f_SDSS','Vel','logL','Type','Q','zr','zh'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#ju2013 = ju2013[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#ju2013.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']
#


ju2013

,SDSS,Ju2013_flag,z,M_abs,log_10M_BH,Deltat,DeltaV_1,DeltaV_2,Error,r_max,Last_Observation,Name1,Name2,z1,z2,z1_type,z2_type,RA,Dec,RA_deg,Dec_deg,RA2,Dec2,RA2_deg,Dec2_deg,Equinox,Coordinate_waveband,Coordinate_Source,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Sep,delta_z,dV,Selection Method,Confirmation Method,Paper(s),BibCode(s),DOI(s),Notes
0,J032223.02-000803.5,NaN,0.62,-22.8,8.2827,0.9123,780,760,0.22,0.032,10-20-2001,J032223.02-000803.5,J032223.02-000803.5,0.62,0.62,spec,spec,03:22:23.02,-00:08:03.5,50.595917,-0.134306,03:22:23.02,-00:08:03.5,50.595917,-0.134306,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Ju+2013,2013ApJ...777...44J,https://doi.org/10.1088/0004-637X/777/1/44,
1,J002444.11+003221.4,*,0.40,-24.3,9.5618,0.2301,328,380,2.76,0.102,12-22-2000,J002444.11+003221.4,J002444.11+003221.4,0.40,0.40,spec,spec,00:24:44.11,+00:32:21.4,6.183792,0.539278,00:24:44.11,+00:32:21.4,6.183792,0.539278,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Ju+2013,2013ApJ...777...44J,https://doi.org/10.1088/0004-637X/777/1/44,
2,J095656.42+535023.2,b+m*,0.61,-23.1,8.2944,6.1589,360,300,1.74,0.127,03-05-2008,J095656.42+535023.2,J095656.42+535023.2,0.61,0.61,spec,spec,09:56:56.42,+53:50:23.2,149.235083,53.839778,09:56:56.42,+53:50:23.2,149.235083,53.839778,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Ju+2013,2013ApJ...777...44J,https://doi.org/10.1088/0004-637X/777/1/44,
3,J161609.50+434146.8,m,0.49,-22.8,8.1696,0.2000,330,250,5.10,0.021,06-18-2002,J161609.50+434146.8,J161609.50+434146.8,0.49,0.49,spec,spec,16:16:09.50,+43:41:46.8,244.039583,43.696333,16:16:09.50,+43:41:46.8,244.039583,43.696333,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Ju+2013,2013ApJ...777...44J,https://doi.org/10.1088/0004-637X/777/1/44,
4,J075700.70+424814.5,NaN,1.17,-24.7,9.1311,0.0192,310,290,2.12,0.020,11-28-2000,J075700.70+424814.5,J075700.70+424814.5,1.17,1.17,spec,spec,07:57:00.70,+42:48:14.5,119.252917,42.804028,07:57:00.70,+42:48:14.5,119.252917,42.804028,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Ju+2013,2013ApJ...777...44J,https://doi.org/10.1088/0004-637X/777/1/44,
5,J093502.54+433110.7,m,0.46,-25.6,9.3425,0.9452,290,270,2.10,0.181,01-31-2003,J093502.54+433110.7,J093502.54+433110.7,0.46,0.46,spec,spec,09:35:02.54,+43:31:10.7,143.760583,43.519639,09:35:02.54,+43:31:10.7,143.760583,43.519639,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Ju+2013,2013ApJ...777...44J,https://doi.org/10.1088/0004-637X/777/1/44,
6,J004918.98+002609.4,*,1.94,-26.1,9.3148,0.2767,-260,-290,5.65,0.096,01-04-2001,J004918.98+002609.4,J004918.98+002609.4,1.94,1.94,spec,spec,00:49:18.98,+00:26:09.4,12.329083,0.435944,00:49:18.98,+00:26:09.4,12.329083,0.435944,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Ju+2013,2013ApJ...777...44J,https://doi.org/10.1088/0004-637X/777/1/44,


In [10]:
# Loading in here the results from Wang+2017, a follow-up work for Ju+2013

wang2017t1 = pd.read_csv('Tables/Wang2017/Wang2017.csv',sep=',')

objs = wang2017t1['SDSSID'].tolist()

for index, row in ju2013.iterrows():
    if row['SDSS'] in objs:
        #the_whills.at[index, 'System Type']='Dual AGN'
        ju2013.at[index, 'Paper(s)'] += ' ; Wang+2017'
        ju2013.at[index, 'BibCode(s)'] += ' ; 2017ApJ...834..129W' 
        ju2013.at[index, 'DOI(s)'] += ' ; https://doi.org/10.3847/1538-4357/834/2/129'
        ju2013.at[index, 'Notes ']+=' Wang+ re-examine with additional epoch spectra. They rule out the binary hypothesis for this object. The lineo-fo-sight acceleration did not change between observations as expected.'

# ******************************
# --> These are all from the Mg II cross correlation only. We'll need to load in that table from Ju+2013!
# ******************************
        
        
wang2017 = pd.read_csv('Tables/Wang2017/Wang2017_t3.csv', sep=',')

wang2017['Name1'] = wang2017['SDSSID']
wang2017['Name2'] = wang2017['SDSSID']
wang2017['z1'] = wang2017['z']
wang2017['z2'] = wang2017['z']
wang2017['z1_type'] = "spec"
wang2017['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(wang2017,wang2017['Designation'])

wang2017['Coordinates'] = wang2017['SDSSID']#.str.slice(start=1) # Stripping the J
wang2017['RA_test'] = wang2017['Coordinates'].str.slice(start=1, stop=10) # Stripping the DEC parts 
wang2017['Dec_test'] = wang2017['Coordinates'].str.slice(start=10, stop=19) # Stripping the RA parts
wang2017['RA'] = wang2017['RA_test'].str.slice(start=0, stop=2)+":"+wang2017['RA_test'].str.slice(start=2, stop=4)+":"+wang2017['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
wang2017['Dec'] = wang2017['Dec_test'].str.slice(start=0, stop=3)+":"+wang2017['Dec_test'].str.slice(start=3, stop=5)+":"+wang2017['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
wang2017.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)

# Converting the coordinates
coordconvert = SkyCoord(ra = wang2017['RA'], dec = wang2017['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
wang2017['RA_deg'] = coordconvert.ra.degree
wang2017['Dec_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
wang2017['RA2'] = wang2017['RA']
wang2017['Dec2'] = wang2017['Dec']

wang2017['RA2_deg'] = wang2017['RA_deg']
wang2017['Dec2_deg'] = wang2017['Dec_deg']

# Adding details about the coordinates
wang2017['Equinox'] = "J2000"
wang2017['Coordinate_waveband'] = "Optical"
wang2017['Coordinate_Source'] = "SDSS"

wang2017['System Type'] = 'Binary AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
wang2017['Brightness1'] = -100
wang2017['Brightness_band1'] = -100
wang2017['Brightness_type1'] = -100

wang2017['Brightness2'] = -100
wang2017['Brightness_band2'] = -100
wang2017['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
#wang2017['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit


#wang2017['Sep(kpc)'] = wang2017['Sep']*((cosmo.arcsec_per_kpc_proper(wang2017['z']))**(-1))


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
wang2017['delta_z'] = wang2017['z1']-wang2017['z2']
wang2017['dV'] = (2.99e+5)*((1+wang2017['z1'])**2 - (1+wang2017['z2'])**2)/((1+wang2017['z1'])**2+(1+wang2017['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
wang2017['Selection Method'] = "Velocity Offset Emission Lines" #DPSELs
wang2017['Confirmation Method'] = "-99"
wang2017['Paper(s)'] = "Wang+2017"
wang2017['BibCode(s)'] = "2017ApJ...834..129W"
wang2017['DOI(s)'] = "https://doi.org/ 10.3847/1538-4357/834/2/129"

## And dropping any columns that we don't need....
#wang2017.drop(labels=['SDSS','f_SDSS','Vel','logL','Type','Q','zr','zh'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#wang2017 = wang2017[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#wang2017.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']
#

# Eventually we need to go back and flag the single object that is favored by them and flag the others as disfavored.

wang2017



,SDSSID,z,log10MBH,delta_t,delta_V,aLoS,Name1,Name2,z1,z2,z1_type,z2_type,RA,Dec,RA_deg,Dec_deg,RA2,Dec2,RA2_deg,Dec2_deg,Equinox,Coordinate_waveband,Coordinate_Source,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,delta_z,dV,Selection Method,Confirmation Method,Paper(s),BibCode(s),DOI(s)
0,J002127.88+010420.2,1.82,9.81,9.81,-345,-35,J002127.88+010420.2,J002127.88+010420.2,1.82,1.82,spec,spec,00:21:27.88,+01:04:20.2,5.366167,1.072278,00:21:27.88,+01:04:20.2,5.366167,1.072278,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,Velocity Offset Emission Lines,-99,Wang+2017,2017ApJ...834..129W,https://doi.org/ 10.3847/1538-4357/834/2/129
1,J003333.61-001858.1,0.69,9.27,10.03,293,29,J003333.61-001858.1,J003333.61-001858.1,0.69,0.69,spec,spec,00:33:33.61,-00:18:58.1,8.390042,-0.316139,00:33:33.61,-00:18:58.1,8.390042,-0.316139,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,Velocity Offset Emission Lines,-99,Wang+2017,2017ApJ...834..129W,https://doi.org/ 10.3847/1538-4357/834/2/129
2,J014822.62+132142.6,0.88,9.36,10.33,-345,-33,J014822.62+132142.6,J014822.62+132142.6,0.88,0.88,spec,spec,01:48:22.62,+13:21:42.6,27.094250,13.361833,01:48:22.62,+13:21:42.6,27.094250,13.361833,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,Velocity Offset Emission Lines,-99,Wang+2017,2017ApJ...834..129W,https://doi.org/ 10.3847/1538-4357/834/2/129
3,J091344.40+150935.1,0.94,9.45,5.28,-276,-52,J091344.40+150935.1,J091344.40+150935.1,0.94,0.94,spec,spec,09:13:44.40,+15:09:35.1,138.435000,15.159750,09:13:44.40,+15:09:35.1,138.435000,15.159750,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,Velocity Offset Emission Lines,-99,Wang+2017,2017ApJ...834..129W,https://doi.org/ 10.3847/1538-4357/834/2/129
4,J095929.88+633359.8,1.85,10.74,11.14,-328,-29,J095929.88+633359.8,J095929.88+633359.8,1.85,1.85,spec,spec,09:59:29.88,+63:33:59.8,149.874500,63.566611,09:59:29.88,+63:33:59.8,149.874500,63.566611,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,Velocity Offset Emission Lines,-99,Wang+2017,2017ApJ...834..129W,https://doi.org/ 10.3847/1538-4357/834/2/129
5,J103623.66+152733.3,1.92,9.95,5.02,328,65,J103623.66+152733.3,J103623.66+152733.3,1.92,1.92,spec,spec,10:36:23.66,+15:27:33.3,159.098583,15.459250,10:36:23.66,+15:27:33.3,159.098583,15.459250,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,Velocity Offset Emission Lines,-99,Wang+2017,2017ApJ...834..129W,https://doi.org/ 10.3847/1538-4357/834/2/129
6,J105611.27+170827.5,1.33,9.62,5.93,-293,-49,J105611.27+170827.5,J105611.27+170827.5,1.33,1.33,spec,spec,10:56:11.27,+17:08:27.5,164.046958,17.140972,10:56:11.27,+17:08:27.5,164.046958,17.140972,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,Velocity Offset Emission Lines,-99,Wang+2017,2017ApJ...834..129W,https://doi.org/ 10.3847/1538-4357/834/2/129
7,J110038.79+282036.1,1.79,9.60,7.06,258,36,J110038.79+282036.1,J110038.79+282036.1,1.79,1.79,spec,spec,11:00:38.79,+28:20:36.1,165.161625,28.343361,11:00:38.79,+28:20:36.1,165.161625,28.343361,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,Velocity Offset Emission Lines,-99,Wang+2017,2017ApJ...834..129W,https://doi.org/ 10.3847/1538-4357/834/2/129
8,J125238.51+115557.1,1.85,9.64,6.87,310,45,J125238.51+115557.1,J125238.51+115557.1,1.85,1.85,spec,spec,12:52:38.51,+11:55:57.1,193.160458,11.932528,12:52:38.51,+11:55:57.1,193.160458,11.932528,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,0.0,0.0,Velocity Offset Emission Lines,-99,Wang+2017,2017ApJ...834..129W,https://doi.org/ 10.3847/1538-4357/834/2/129
9,J133615.79+495529.0,1.50,9.62,8.13,-207,-25,J133615.79+495529.0,J133615.79+495529.0,1.50,1.50,spec,spec,13:36:15.79,+49:55:29.0,204.065792,49.924722,13:36:15.79,+49:55:29.0,204.065792,49.924722,J2000,Optical,SDS

In [11]:
ju2013

,SDSS,Ju2013_flag,z,M_abs,log_10M_BH,Deltat,DeltaV_1,DeltaV_2,Error,r_max,Last_Observation,Name1,Name2,z1,z2,z1_type,z2_type,RA,Dec,RA_deg,Dec_deg,RA2,Dec2,RA2_deg,Dec2_deg,Equinox,Coordinate_waveband,Coordinate_Source,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Sep,delta_z,dV,Selection Method,Confirmation Method,Paper(s),BibCode(s),DOI(s),Notes
0,J032223.02-000803.5,NaN,0.62,-22.8,8.2827,0.9123,780,760,0.22,0.032,10-20-2001,J032223.02-000803.5,J032223.02-000803.5,0.62,0.62,spec,spec,03:22:23.02,-00:08:03.5,50.595917,-0.134306,03:22:23.02,-00:08:03.5,50.595917,-0.134306,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Ju+2013,2013ApJ...777...44J,https://doi.org/10.1088/0004-637X/777/1/44,
1,J002444.11+003221.4,*,0.40,-24.3,9.5618,0.2301,328,380,2.76,0.102,12-22-2000,J002444.11+003221.4,J002444.11+003221.4,0.40,0.40,spec,spec,00:24:44.11,+00:32:21.4,6.183792,0.539278,00:24:44.11,+00:32:21.4,6.183792,0.539278,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Ju+2013,2013ApJ...777...44J,https://doi.org/10.1088/0004-637X/777/1/44,
2,J095656.42+535023.2,b+m*,0.61,-23.1,8.2944,6.1589,360,300,1.74,0.127,03-05-2008,J095656.42+535023.2,J095656.42+535023.2,0.61,0.61,spec,spec,09:56:56.42,+53:50:23.2,149.235083,53.839778,09:56:56.42,+53:50:23.2,149.235083,53.839778,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Ju+2013,2013ApJ...777...44J,https://doi.org/10.1088/0004-637X/777/1/44,
3,J161609.50+434146.8,m,0.49,-22.8,8.1696,0.2000,330,250,5.10,0.021,06-18-2002,J161609.50+434146.8,J161609.50+434146.8,0.49,0.49,spec,spec,16:16:09.50,+43:41:46.8,244.039583,43.696333,16:16:09.50,+43:41:46.8,244.039583,43.696333,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Ju+2013,2013ApJ...777...44J,https://doi.org/10.1088/0004-637X/777/1/44,
4,J075700.70+424814.5,NaN,1.17,-24.7,9.1311,0.0192,310,290,2.12,0.020,11-28-2000,J075700.70+424814.5,J075700.70+424814.5,1.17,1.17,spec,spec,07:57:00.70,+42:48:14.5,119.252917,42.804028,07:57:00.70,+42:48:14.5,119.252917,42.804028,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Ju+2013,2013ApJ...777...44J,https://doi.org/10.1088/0004-637X/777/1/44,
5,J093502.54+433110.7,m,0.46,-25.6,9.3425,0.9452,290,270,2.10,0.181,01-31-2003,J093502.54+433110.7,J093502.54+433110.7,0.46,0.46,spec,spec,09:35:02.54,+43:31:10.7,143.760583,43.519639,09:35:02.54,+43:31:10.7,143.760583,43.519639,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Ju+2013,2013ApJ...777...44J,https://doi.org/10.1088/0004-637X/777/1/44,
6,J004918.98+002609.4,*,1.94,-26.1,9.3148,0.2767,-260,-290,5.65,0.096,01-04-2001,J004918.98+002609.4,J004918.98+002609.4,1.94,1.94,spec,spec,00:49:18.98,+00:26:09.4,12.329083,0.435944,00:49:18.98,+00:26:09.4,12.329083,0.435944,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Double-Peaked Optical Spectroscopic Emission L...,-99,Ju+2013,2013ApJ...777...44J,https://doi.org/10.1088/0004-637X/777/1/44,


In [21]:
# Loading in Liu+2014 (Paper II of their series, wehre Shen+2013 was Paper I)

# NOTE: The SDSS designations change between Tables 1 and 2 in Liu+2014. 
# This very likely means that the designations in Gou+ will only match one of these tables (or neither of them)
# I am not going to go through and fix this now. 

# For now, I am ONLY loading in a table of the specific binary candidates. I am not loading in their full sample!
#liu2014 = ((Table.read('Tables/XLiu2014/table1.dat', readme = 'Tables/XLiu2014/ReadMe', format='ascii.cds')).to_pandas())#.drop(columns=['---'])
liu2014 = pd.read_csv('Tables/XLiu2014/table2.csv', sep=',')
# Limiting to only the systems they flagged as binary candidates now....
liu2014 = liu2014t2[liu2014t2['Category']=='1']

liu2014['Name1'] = liu2014['SDSS Designation']
liu2014['Name2'] = liu2014['SDSS Designation']
liu2014['z'] = -99

liu2014[['SDSS Designation']=='082930.60+272822.7', 'z']=0.3211
liu2014[['SDSS Designation']=='084716.04+373218.1', 'z']=0.4534
liu2014[['SDSS Designation']=='085237.02+200411.0', 'z']=0.4615
liu2014[['SDSS Designation']=='092837.98+602521.0', 'z']=0.2959
liu2014[['SDSS Designation']=='103059.09+310255.8', 'z']=0.1781
liu2014[['SDSS Designation']=='110051.02+170934.3', 'z']=0.3476
liu2014[['SDSS Designation']=='111230.90+181311.4', 'z']=0.1952
liu2014[['SDSS Designation']=='130534.49+181932.9', 'z']=0.1178
liu2014[['SDSS Designation']=='134548.50+114443.5', 'z']=0.1264

liu2014['z1'] = liu2014['z']
liu2014['z2'] = liu2014['z']
liu2014['z1_type'] = "spec"
liu2014['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(liu2014,liu2014['Designation'])

liu2014['Coordinates'] = liu2014['SDSS Designation']#.str.slice(start=1) # Stripping the J
liu2014['RA_test'] = liu2014['Coordinates'].str.slice(start=0, stop=9) # Stripping the DEC parts 
liu2014['Dec_test'] = liu2014['Coordinates'].str.slice(start=9, stop=19) # Stripping the RA parts
liu2014['RA'] = liu2014['RA_test'].str.slice(start=0, stop=2)+":"+liu2014['RA_test'].str.slice(start=2, stop=4)+":"+liu2014['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
liu2014['Dec'] = liu2014['Dec_test'].str.slice(start=0, stop=3)+":"+liu2014['Dec_test'].str.slice(start=3, stop=5)+":"+liu2014['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
liu2014.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)

# Converting the coordinates
coordconvert = SkyCoord(ra = liu2014['RA'], dec = liu2014['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
liu2014['RA_deg'] = coordconvert.ra.degree
liu2014['Dec_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
liu2014['RA2'] = liu2014['RA']
liu2014['Dec2'] = liu2014['Dec']

liu2014['RA2_deg'] = liu2014['RA_deg']
liu2014['Dec2_deg'] = liu2014['Dec_deg']

# Adding details about the coordinates
liu2014['Equinox'] = "J2000"
liu2014['Coordinate_waveband'] = "Optical"
liu2014['Coordinate_Source'] = "SDSS"

liu2014['System Type'] = 'Binary AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
liu2014['Brightness1'] = -100
liu2014['Brightness_band1'] = -100
liu2014['Brightness_type1'] = -100

liu2014['Brightness2'] = -100
liu2014['Brightness_band2'] = -100
liu2014['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
liu2014['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit

#liu2014['Sep(kpc)'] = liu2014['Sep']*((cosmo.arcsec_per_kpc_proper(liu2014['z']))**(-1))

# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
liu2014['delta_z'] = liu2014['z1']-liu2014['z2']
liu2014['dV'] = (2.99e+5)*((1+liu2014['z1'])**2 - (1+liu2014['z2'])**2)/((1+liu2014['z1'])**2+(1+liu2014['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
liu2014['Selection Method'] = "Extreme Velocity Offsets / Radial Velocity Shifts" #DPSELs
liu2014['Confirmation Method'] = "-99"
liu2014['Paper(s)'] = "Liu+2014"
liu2014['BibCode(s)'] = "2014ApJ...789..140L"
liu2014['DOI(s)'] = "https://doi.org/10.1088/0004-637X/789/2/140"

## And dropping any columns that we don't need....
#liu2014.drop(labels=['SDSS','f_SDSS','Vel','logL','Type','Q','zr','zh'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#liu2014 = liu2014[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#liu2014.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']
#

#liu2014t2 = pd.read_csv('Tables/XLiu2014/table2.csv', sep=',')
# Limiting to only the systems they flagged as binary candidates now....
#liu2014t2 = liu2014t2[liu2014t2['Category']=='1']

# a basic .isin check doesn't work because the designations changed between tables 1 and 2...
# Manually correcting this now...
# Adding the DOI, author, and bibcode info to all of the Liu+2010 rows here in the matches table...
#liu2014 = liu2014[liu2014['SDSS'].isin(liu2014t2['SDSS Designation'])]

liu2014


,No.,SDSS Designation,r_mag,Spec,UT,t_exp(s),S/N(pixel-1),delta_t(yr),Vccf(km s-1),accf(kms-1 yr-1),Category,Name1,Name2,z,"(False, z)",z1,z2,z1_type,z2_type,RA,Dec,RA_deg,Dec_deg,RA2,Dec2,RA2_deg,Dec2_deg,Equinox,Coordinate_waveband,Coordinate_Source,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Sep,delta_z,dV,Selection Method,Confirmation Method,Paper(s),BibCode(s),DOI(s)
13,14,082930.60+272822.7,18.10,FAST,111104,25200,23,6.24,-213+129-127,-34+21-20,1,082930.60+272822.7,082930.60+272822.7,-99,0.1264,-99,-99,spec,spec,08:29:30.60,+27:28:22.7,127.377500,27.472972,08:29:30.60,+27:28:22.7,127.377500,27.472972,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0,0.0,Extreme Velocity Offsets / Radial Velocity Shifts,-99,Liu+2014,2014ApJ...789..140L,https://doi.org/10.1088/0004-637X/789/2/140
14,15,084716.04+373218.1,18.45,DIS,100416,2700,17,5.76,55+46-39,10+8-7,1,084716.04+373218.1,084716.04+373218.1,-99,0.1264,-99,-99,spec,spec,08:47:16.04,+37:32:18.1,131.816833,37.538361,08:47:16.04,+37:32:18.1,131.816833,37.538361,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0,0.0,Extreme Velocity Offsets / Radial Velocity Shifts,-99,Liu+2014,2014ApJ...789..140L,https://doi.org/10.1088/0004-637X/789/2/140
15,16,085237.02+200411.0,18.10,DIS,100416,2700,17,3.12,-296+162-120,-95+52-38,1,085237.02+200411.0,085237.02+200411.0,-99,0.1264,-99,-99,spec,spec,08:52:37.02,+20:04:11.0,133.154250,20.069722,08:52:37.02,+20:04:11.0,133.154250,20.069722,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0,0.0,Extreme Velocity Offsets / Radial Velocity Shifts,-99,Liu+2014,2014ApJ...789..140L,https://doi.org/10.1088/0004-637X/789/2/140
20,21,092837.98+602521.0,17.01,FAST,120313,5400,25,8.87,220+154-207,24+18-23,1,092837.98+602521.0,092837.98+602521.0,-99,0.1264,-99,-99,spec,spec,09:28:37.98,+60:25:21.0,142.158250,60.422500,09:28:37.98,+60:25:21.0,142.158250,60.422500,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0,0.0,Extreme Velocity Offsets / Radial Velocity Shifts,-99,Liu+2014,2014ApJ...789..140L,https://doi.org/10.1088/0004-637X/789/2/140
24,25,103059.09+310255.8,16.77,FAST,120119,1800,13,5.97,289+130-133,48±22,1,103059.09+310255.8,103059.09+310255.8,-99,0.1264,-99,-99,spec,spec,10:30:59.09,+31:02:55.8,157.746208,31.048833,10:30:59.09,+31:02:55.8,157.746208,31.048833,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0,0.0,Extreme Velocity Offsets / Radial Velocity Shifts,-99,Liu+2014,2014ApJ...789..140L,https://doi.org/10.1088/0004-637X/789/2/140
26,27,110051.02+170934.3,18.48,DIS,110601,3600,21,3.20,-269+92-94,-84±29,1,110051.02+170934.3,110051.02+170934.3,-99,0.1264,-99,-99,spec,spec,11:00:51.02,+17:09:34.3,165.212583,17.159528,11:00:51.02,+17:09:34.3,165.212583,17.159528,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0,0.0,Extreme Velocity Offsets / Radial Velocity Shifts,-99,Liu+2014,2014ApJ...789..140L,https://doi.org/10.1088/0004-637X/789/2/140
27,28,111230.90+181311.4,18.13,FAST,120101,9000,17,4.10,-124+88-86,-30+22-21,1,111230.90+181311.4,111230.90+181311.4,-99,0.1264,-99,-99,spec,spec,11:12:30.90,+18:13:11.4,168.128750,18.219833,11:12:30.90,+18:13:11.4,168.128750,18.219833,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0,0.0,Extreme Velocity Offsets / Radial Velocity Shifts,-99,Liu+2014,2014ApJ...789..140L,https://doi.org/10.1088/0004-637X/789/2/140
33,34,130534.49+181932.9,16.68,DIS,100412,2700,24,1.98,236+79-76,119±39,1,130534.49+181932.9,130534.49+181932.9,-99,0.1264,-99,-99,spec,spec,13:05:34.49,+18:19:32.9,196.393708,18.325806,13:05:34.49,+18:19:32.9,196.393708,18.325806,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0,0.0,Extreme Velocity Offsets / Radial Velocity Shifts,-99,Liu+2014,2014ApJ...789..140L,https://doi.org/10.1088/0004-637X/789/2/140
35,36,134548.50+114443.5,17.03,FAST,120419,5400,16,

In [33]:
guo2019

,No.,Name,MJD1,MJD2,MJD3,MJD4,V2,2.5σ,1σ,V3,2.5σ.1,1σ.1,V4,2.5σ.2,1σ.2,Category
0,1,032213.89+005513.4,51901,53358,57252,...,-44,±25,±9,40,±7,±3,...,...,...,2
1,2,082930.60+272822.7,51781,55869,57463,...,-190,±79,±31,-685,±89,±35,...,...,...,2
2,3,084716.04+373218.1,52323,55302,57452,57463,98,±31,±12,123,±21,±8,149,±22,±8,1
3,4,085237.02+200411.0,53680,55302,55955,57461,-162,±100,±39,-368,±76,±30,-186,±57,±22,1
4,5,092837.98+602521.0,51909,55999,57461,...,423,±128,±51,312,±133,±51,...,...,...,1
5,6,103059.09+310255.8,53440,55945,57464,...,290,±135,±52,195,±120,±46,...,...,...,2
6,7,110051.02+170934.3,54176,55713,57464,...,-263,±51,±19,-451,±97,±37,...,...,...,2
7,8,111230.90+181311.4,54179,55927,57464,...,-240,±95,±37,-394,±82,±30,...,...,...,1
8,9,141020.57+364322.7,53143,54590,57437,...,-353,±109,±43,-245,±46,±19,...,...,...,3
9,10,153705.95+005522.8,51663,54562,57252,57437,126,±70,±27,201,±60,±23,257,±94,±37,1


In [42]:
#Now for Gou+2019
guo2019 = pd.read_csv('Tables/Gou2019/Gou2019t2.csv', sep=',')

guo2019objs = guo2019['Name'].tolist()


# I think all we need to do here is append the Gou+2019 paper information for now. If need be, we can go back for \
# more later on...

for index, row in shen2013.iterrows():
    if row['Designation'] in guo2019objs:
        shen2013.at[index, 'Paper(s)'] += ' ; Guo+2019'
        shen2013.at[index, 'BibCode(s)'] += ' ; 2019MNRAS.482.3288G' 
        shen2013.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1093/mnras/sty2920'

for index, row in liu2014.iterrows():
    if row['SDSS Designation'] in guo2019objs:
        liu2014.at[index, 'Paper(s)'] += ' ; Guo+2019'
        liu2014.at[index, 'BibCode(s)'] += ' ; 2019MNRAS.482.3288G' 
        liu2014.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1093/mnras/sty2920'

# Need to add notes about which one Guo+2019 considered to be thr remainding best candidates....


In [43]:
shen2013

,Designation,z_sys,logM_1(M_sun),R_BLR(pc),E1Plate,E1Fiber,E1MJD,E2Plate,E2Fiber,E2MJD,Deltat(days),$V_{\rm ccf}^*$(km s^-1),$V_{\rm ccf}^*$(km s^-1).1,$V_{\rm ccf}^*$(km s^-1).2,$a_{\rm ccf}^*$(km s^-1 yr^-1),$a_{\rm ccf}^*$(km s^-1 yr^-1).1,$a_{\rm ccf}^*$(km s^-1 yr^-1).2,Category,Name1,Name2,z1,z2,z1_type,z2_type,RA,Dec,RA_deg,Dec_deg,RA2,Dec2,RA2_deg,Dec2_deg,Equinox,Coordinate_waveband,Coordinate_Source,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Sep,delta_z,dV,Selection Method,Confirmation Method,Paper(s),BibCode(s),DOI(s)
3,030100.23+000429.3,0.486,8.50,0.056,411,265,51817,709,477,52205,261,420,163,545,588,229.0,762.0,1.0,030100.23+000429.3,030100.23+000429.3,0.486,0.486,spec,spec,03:01:00.23,+00:04:29.3,45.250958,0.074806,03:01:00.23,+00:04:29.3,45.250958,0.074806,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Extreme Velocity Offsets / Radial Velocity Shifts,-99,Shen+2013,2013ApJ...775...49S,https://doi:10.1088/0004-637X/775/1/49
7,032213.89+005513.4,0.185,8.04,0.068,414,341,51901,1181,329,53358,1229,48,21,78,14,6.0,23.0,1.0,032213.89+005513.4,032213.89+005513.4,0.185,0.185,spec,spec,03:22:13.89,+00:55:13.4,50.557875,0.920389,03:22:13.89,+00:55:13.4,50.557875,0.920389,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Extreme Velocity Offsets / Radial Velocity Shifts,-99,Shen+2013 ; Guo+2019,2013ApJ...775...49S ; 2019MNRAS.482.3288G,https://doi:10.1088/0004-637X/775/1/49 ; https...
13,122909.52-003530.0,0.450,8.84,0.056,289,150,51990,2568,32,54153,1491,324,197,453,79,48.0,111.0,1.0,122909.52-003530.0,122909.52-003530.0,0.450,0.450,spec,spec,12:29:09.52,-00:35:30.0,187.289667,-0.591667,12:29:09.52,-00:35:30.0,187.289667,-0.591667,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Extreme Velocity Offsets / Radial Velocity Shifts,-99,Shen+2013,2013ApJ...775...49S,https://doi:10.1088/0004-637X/775/1/49
18,141020.57+364322.7,0.450,8.44,0.053,1643,359,53143,2931,496,54590,998,227,142,309,83,52.0,113.0,1.0,141020.57+364322.7,141020.57+364322.7,0.450,0.450,spec,spec,14:10:20.57,+36:43:22.7,212.585708,36.722972,14:10:20.57,+36:43:22.7,212.585708,36.722972,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Extreme Velocity Offsets / Radial Velocity Shifts,-99,Shen+2013 ; Guo+2019,2013ApJ...775...49S ; 2019MNRAS.482.3288G,https://doi:10.1088/0004-637X/775/1/49 ; https...
19,153705.95+005522.8,0.137,7.63,0.038,2955,289,54562,315,419,51663,2550,110,43,181,15,6.0,26.0,1.0,153705.95+005522.8,153705.95+005522.8,0.137,0.137,spec,spec,15:37:05.95,+00:55:22.8,234.274792,0.923000,15:37:05.95,+00:55:22.8,234.274792,0.923000,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Extreme Velocity Offsets / Radial Velocity Shifts,-99,Shen+2013 ; Guo+2019,2013ApJ...775...49S ; 2019MNRAS.482.3288G,https://doi:10.1088/0004-637X/775/1/49 ; https...
21,155053.16+052112.1,0.110,8.96,0.039,1822,308,53172,2951,326,54592,1278,-1034,-1183,-882,-295,-338.0,-251.0,1.0,155053.16+052112.1,155053.16+052112.1,0.110,0.110,spec,spec,15:50:53.16,+05:21:12.1,237.721500,5.353361,15:50:53.16,+05:21:12.1,237.721500,5.353361,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Extreme Velocity Offsets / Radial Velocity Shifts,-99,Shen+2013 ; Guo+2019,2013ApJ...775...49S ; 2019MNRAS.482.3288G,https://doi:10.1088/0004-637X/775/1/49 ; https...
28,234932.77-003645.8,0.279,8.26,0.068,386,215,51788,684,261,52523,574,-68,-118,-17,-43,-75.0,-11.0,1.0,234932.77-003645.8,234932.77-003645.8,0.279,0.279,spec,spec,23:49:32.77,-00:36:45.8,357.386542,-0.612722,23:49:32.77,-00:36:45.8,357.386542,-0.612722,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0.0,0.0,Extreme Velocity Offsets / Radial Velocity Shifts,-99,Shen+2013 ; Guo+2019,2013ApJ...775...49S ; 2019MNRAS.482.3288G,https://doi:10.1088/0004-637X/775/1/49 ; https...


In [44]:
liu2014

,No.,SDSS Designation,r_mag,Spec,UT,t_exp(s),S/N(pixel-1),delta_t(yr),Vccf(km s-1),accf(kms-1 yr-1),Category,Name1,Name2,z,"(False, z)",z1,z2,z1_type,z2_type,RA,Dec,RA_deg,Dec_deg,RA2,Dec2,RA2_deg,Dec2_deg,Equinox,Coordinate_waveband,Coordinate_Source,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Sep,delta_z,dV,Selection Method,Confirmation Method,Paper(s),BibCode(s),DOI(s)
13,14,082930.60+272822.7,18.10,FAST,111104,25200,23,6.24,-213+129-127,-34+21-20,1,082930.60+272822.7,082930.60+272822.7,-99,0.1264,-99,-99,spec,spec,08:29:30.60,+27:28:22.7,127.377500,27.472972,08:29:30.60,+27:28:22.7,127.377500,27.472972,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0,0.0,Extreme Velocity Offsets / Radial Velocity Shifts,-99,Liu+2014 ; Guo+2019,2014ApJ...789..140L ; 2019MNRAS.482.3288G,https://doi.org/10.1088/0004-637X/789/2/140 ; ...
14,15,084716.04+373218.1,18.45,DIS,100416,2700,17,5.76,55+46-39,10+8-7,1,084716.04+373218.1,084716.04+373218.1,-99,0.1264,-99,-99,spec,spec,08:47:16.04,+37:32:18.1,131.816833,37.538361,08:47:16.04,+37:32:18.1,131.816833,37.538361,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0,0.0,Extreme Velocity Offsets / Radial Velocity Shifts,-99,Liu+2014 ; Guo+2019,2014ApJ...789..140L ; 2019MNRAS.482.3288G,https://doi.org/10.1088/0004-637X/789/2/140 ; ...
15,16,085237.02+200411.0,18.10,DIS,100416,2700,17,3.12,-296+162-120,-95+52-38,1,085237.02+200411.0,085237.02+200411.0,-99,0.1264,-99,-99,spec,spec,08:52:37.02,+20:04:11.0,133.154250,20.069722,08:52:37.02,+20:04:11.0,133.154250,20.069722,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0,0.0,Extreme Velocity Offsets / Radial Velocity Shifts,-99,Liu+2014 ; Guo+2019,2014ApJ...789..140L ; 2019MNRAS.482.3288G,https://doi.org/10.1088/0004-637X/789/2/140 ; ...
20,21,092837.98+602521.0,17.01,FAST,120313,5400,25,8.87,220+154-207,24+18-23,1,092837.98+602521.0,092837.98+602521.0,-99,0.1264,-99,-99,spec,spec,09:28:37.98,+60:25:21.0,142.158250,60.422500,09:28:37.98,+60:25:21.0,142.158250,60.422500,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0,0.0,Extreme Velocity Offsets / Radial Velocity Shifts,-99,Liu+2014 ; Guo+2019,2014ApJ...789..140L ; 2019MNRAS.482.3288G,https://doi.org/10.1088/0004-637X/789/2/140 ; ...
24,25,103059.09+310255.8,16.77,FAST,120119,1800,13,5.97,289+130-133,48±22,1,103059.09+310255.8,103059.09+310255.8,-99,0.1264,-99,-99,spec,spec,10:30:59.09,+31:02:55.8,157.746208,31.048833,10:30:59.09,+31:02:55.8,157.746208,31.048833,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0,0.0,Extreme Velocity Offsets / Radial Velocity Shifts,-99,Liu+2014 ; Guo+2019,2014ApJ...789..140L ; 2019MNRAS.482.3288G,https://doi.org/10.1088/0004-637X/789/2/140 ; ...
26,27,110051.02+170934.3,18.48,DIS,110601,3600,21,3.20,-269+92-94,-84±29,1,110051.02+170934.3,110051.02+170934.3,-99,0.1264,-99,-99,spec,spec,11:00:51.02,+17:09:34.3,165.212583,17.159528,11:00:51.02,+17:09:34.3,165.212583,17.159528,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0,0.0,Extreme Velocity Offsets / Radial Velocity Shifts,-99,Liu+2014 ; Guo+2019,2014ApJ...789..140L ; 2019MNRAS.482.3288G,https://doi.org/10.1088/0004-637X/789/2/140 ; ...
27,28,111230.90+181311.4,18.13,FAST,120101,9000,17,4.10,-124+88-86,-30+22-21,1,111230.90+181311.4,111230.90+181311.4,-99,0.1264,-99,-99,spec,spec,11:12:30.90,+18:13:11.4,168.128750,18.219833,11:12:30.90,+18:13:11.4,168.128750,18.219833,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,3,0,0.0,Extreme Velocity Offsets / Radial Velocity Shifts,-99,Liu+2014 ; Guo+2019,2014ApJ...789..140L ; 2019MNRAS.482.3288G,https://doi.org/10.1088/0004-637X/789/2/140 ; ...
33,34,130534.49+181932.9,16.68,DIS,100412,2700,24,1.98,236+79-76,119±39,1,130534.49+181932.9,130534.49+181932.9,-99,0.1264,-99,-99,spec,spec,13:05:34.49,+18:19:32.9,196.393708,18.325806,13:05:34.49,+18:19:32.9,196.

In [4]:
# This needs to get moved to a miscellaneous notebook
# Loading in Mezcua+2014 here

mezcua2014 = pd.read_csv('Tables/Mezcua2014/Mezcua2014.csv', sep=',')

# We need to gather coordinates, redshifts, and other information....


#mezcua2014['Name1'] = mezcua2014['SDSS Designation']
#mezcua2014['Name2'] = mezcua2014['SDSS Designation']
#mezcua2014['z'] = -99

#mezcua2014['z1'] = mezcua2014['z']
#mezcua2014['z2'] = mezcua2014['z']
#mezcua2014['z1_type'] = "spec"
#mezcua2014['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(mezcua2014,mezcua2014['Designation'])

#mezcua2014['Coordinates'] = mezcua2014['SDSS Designation']#.str.slice(start=1) # Stripping the J
#mezcua2014['RA_test'] = mezcua2014['Coordinates'].str.slice(start=0, stop=9) # Stripping the DEC parts 
#mezcua2014['Dec_test'] = mezcua2014['Coordinates'].str.slice(start=9, stop=19) # Stripping the RA parts
#mezcua2014['RA'] = mezcua2014['RA_test'].str.slice(start=0, stop=2)+":"+mezcua2014['RA_test'].str.slice(start=2, stop=4)+":"+mezcua2014['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
#mezcua2014['Dec'] = mezcua2014['Dec_test'].str.slice(start=0, stop=3)+":"+mezcua2014['Dec_test'].str.slice(start=3, stop=5)+":"+mezcua2014['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
#mezcua2014.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)
#
## Converting the coordinates
#coordconvert = SkyCoord(ra = mezcua2014['RA'], dec = mezcua2014['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
#mezcua2014['RA_deg'] = coordconvert.ra.degree
#mezcua2014['Dec_deg'] = coordconvert.dec.degree
#
## Adding in a second set of coordinates for the 'secondary'
#mezcua2014['RA2'] = mezcua2014['RA']
#mezcua2014['Dec2'] = mezcua2014['Dec']
#
#mezcua2014['RA2_deg'] = mezcua2014['RA_deg']
#mezcua2014['Dec2_deg'] = mezcua2014['Dec_deg']

# Adding details about the coordinates
mezcua2014['Equinox'] = "J2000"
mezcua2014['Coordinate_waveband'] = "Optical"
mezcua2014['Coordinate_Source'] = "SDSS"

mezcua2014['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
mezcua2014['Brightness1'] = -100
mezcua2014['Brightness_band1'] = -100
mezcua2014['Brightness_type1'] = -100

mezcua2014['Brightness2'] = -100
mezcua2014['Brightness_band2'] = -100
mezcua2014['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
mezcua2014['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit

#mezcua2014['Sep(kpc)'] = mezcua2014['Sep']*((cosmo.arcsec_per_kpc_proper(mezcua2014['z']))**(-1))

# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
#mezcua2014['delta_z'] = mezcua2014['z1']-mezcua2014['z2']
#mezcua2014['dV'] = (2.99e+5)*((1+mezcua2014['z1'])**2 - (1+mezcua2014['z2'])**2)/((1+mezcua2014['z1'])**2+(1+mezcua2014['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
mezcua2014['Selection Method'] = "Projected Separation" #DPSELs
mezcua2014['Confirmation Method'] = "-99"
mezcua2014['Paper(s)'] = "Mezcua+2014"
mezcua2014['BibCode(s)'] = "2014ApJ...784...16M"
mezcua2014['DOI(s)'] = "https://doi.org/10.1088/0004-637X/784/1/16"

## And dropping any columns that we don't need....
#mezcua2014.drop(labels=['SDSS','f_SDSS','Vel','logL','Type','Q','zr','zh'],\
#              axis=1, inplace=True)

mezcua2014

,Name,r_arcsec,r_kpc,Equinox,Coordinate_waveband,Coordinate_Source,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Sep,Selection Method,Confirmation Method,Paper(s),BibCode(s),DOI(s)
0,MCG +01-32-049,7.6,0.6,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,3,Projected Separation,-99,Mezcua+2014,2014ApJ...784...16M,https://doi.org/10.1088/0004-637X/784/1/16
1,MCG +02-32-078,8.2,0.5,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,3,Projected Separation,-99,Mezcua+2014,2014ApJ...784...16M,https://doi.org/10.1088/0004-637X/784/1/16
2,MCG +10-19-089,6.6,0.9,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,3,Projected Separation,-99,Mezcua+2014,2014ApJ...784...16M,https://doi.org/10.1088/0004-637X/784/1/16
3,Mrk 22,4.5,0.5,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,3,Projected Separation,-99,Mezcua+2014,2014ApJ...784...16M,https://doi.org/10.1088/0004-637X/784/1/16
4,Mrk 35,4.0,0.3,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,3,Projected Separation,-99,Mezcua+2014,2014ApJ...784...16M,https://doi.org/10.1088/0004-637X/784/1/16
5,Mrk 104,6.2,1.0,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,3,Projected Separation,-99,Mezcua+2014,2014ApJ...784...16M,https://doi.org/10.1088/0004-637X/784/1/16
6,Mrk 116,5.4,0.3,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,3,Projected Separation,-99,Mezcua+2014,2014ApJ...784...16M,https://doi.org/10.1088/0004-637X/784/1/16
7,Mrk 153,4.5,0.8,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,3,Projected Separation,-99,Mezcua+2014,2014ApJ...784...16M,https://doi.org/10.1088/0004-637X/784/1/16
8,Mrk 224,3.9,0.3,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,3,Projected Separation,-99,Mezcua+2014,2014ApJ...784...16M,https://doi.org/10.1088/0004-637X/784/1/16
9,Mrk 731,4.0,0.5,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,3,Projected Separation,-99,Mezcua+2014,2014ApJ...784...16M,https://doi.org/10.1088/0004-637X/784/1/16


In [9]:
# Adding in the recoil candidates from Kim here...

#Col 1: object name. Col 2: redshift. Col 3: SDSS r magnitude. Col 4: IRAS ADDSCAN/SCANPI 60 μm flux. Col 5: IRAS ADDSCAN/SCANPI 100 μm
#flux. Col 6: far-infrared luminosity. Col 7: line-of-sight velocity offset from Hα broad line. Col 8: estimated spatial offsets in arcseconds and kiloparsecs assuming an
#AGN lifetime of 10 Myr. Col 9: line-of-sight velocity offset from Hβ broad line. Col 10: velocity dispersion. Col 11: black hole masses estimated from virial method.
#Col 12: Eddington ratios calculated from virial black hole masses.

kim2016 = pd.read_csv('Tables/Kim2016/Kim2016.csv', sep=',')

kim2016['Name1'] = kim2016['Name']
kim2016['Name2'] = kim2016['Name']
kim2016['z1'] = kim2016['z']
kim2016['z2'] = kim2016['z']
kim2016['z1_type'] = "spec"
kim2016['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
name_to_coords(kim2016,kim2016['Name'])

# Converting the coordinates
coordconvert = SkyCoord(ra = kim2016['RA'], dec = kim2016['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
kim2016['RA_deg'] = coordconvert.ra.degree
kim2016['Dec_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
kim2016['RA2'] = kim2016['RA']
kim2016['Dec2'] = kim2016['Dec']

kim2016['RA2_deg'] = kim2016['RA_deg']
kim2016['Dec2_deg'] = kim2016['Dec_deg']

# Adding details about the coordinates
kim2016['Equinox'] = "J2000"
kim2016['Coordinate_waveband'] = "Optical"
kim2016['Coordinate_Source'] = "SDSS"

kim2016['System Type'] = 'Recoil Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
kim2016['Brightness1'] = -100
kim2016['Brightness_band1'] = -100
kim2016['Brightness_type1'] = -100

kim2016['Brightness2'] = -100
kim2016['Brightness_band2'] = -100
kim2016['Brightness_type2'] = -100


# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
kim2016['Sep'] = kim2016['Spatial Offset (as)'] # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit

#kim2016['Sep(kpc)'] = kim2016['Sep']*((cosmo.arcsec_per_kpc_proper(kim2016['z']))**(-1))

# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
#kim2016['delta_z'] = kim2016['z1']-kim2016['z2']
kim2016['dV'] = kim2016['delta_V(Halpha)'] #(2.99e+5)*((1+kim2016['z1'])**2 - (1+kim2016['z2'])**2)/((1+kim2016['z1'])**2+(1+kim2016['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
kim2016['Selection Method'] = "Offset Broad Lines" #DPSELs
kim2016['Confirmation Method'] = "-99"
kim2016['Paper(s)'] = "Kim+2016"
kim2016['BibCode(s)'] = "2016ApJ...824..122K"
kim2016['DOI(s)'] = "https://doi.org/10.3847/0004-637X/824/2/122"

## And dropping any columns that we don't need....
#kim2016.drop(labels=['SDSS','f_SDSS','Vel','logL','Type','Q','zr','zh'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#kim2016 = kim2016[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#kim2016.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']
#

kim2016


,Name,z,r,f60,f100,logLFIR,delta_V(Halpha),delta_V(Halpha)_err,Spatial Offset (as),Spatial Offset (kpc),delta_V(Hbeta),delta_V(Hbeta)_err,Name1,Name2,z1,z2,z1_type,z2_type,RA,Dec,RA_deg,Dec_deg,RA2,Dec2,RA2_deg,Dec2_deg,Equinox,Coordinate_waveband,Coordinate_Source,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Sep,dV,Selection Method,Confirmation Method,Paper(s),BibCode(s),DOI(s)
0,075819.69+421935.1,0.212,17.01,0.33,0.54,11.98,-154,10,0.46,1.57,-226,25,075819.69+421935.1,075819.69+421935.1,0.212,0.212,spec,spec,07:58:19.69,+42:19:35.1,119.582042,42.326417,07:58:19.69,+42:19:35.1,119.582042,42.326417,J2000,Optical,SDSS,Recoil Candidate,-100,-100,-100,-100,-100,-100,0.46,-154,Offset Broad Lines,-99,Kim+2016,2016ApJ...824..122K,https://doi.org/10.3847/0004-637X/824/2/122
1,080101.41+184840.7,0.140,16.62,0.08,0.62,11.36,-87,10,0.36,0.89,-110,4,080101.41+184840.7,080101.41+184840.7,0.140,0.140,spec,spec,08:01:01.41,+18:48:40.7,120.255875,18.811306,08:01:01.41,+18:48:40.7,120.255875,18.811306,J2000,Optical,SDSS,Recoil Candidate,-100,-100,-100,-100,-100,-100,0.36,-87,Offset Broad Lines,-99,Kim+2016,2016ApJ...824..122K,https://doi.org/10.3847/0004-637X/824/2/122
2,080742.46+375332.1,0.230,18.18,0.06,0.15,11.40,-76,5,0.21,0.78,-105,10,080742.46+375332.1,080742.46+375332.1,0.230,0.230,spec,spec,08:07:42.46,+37:53:32.1,121.926917,37.892250,08:07:42.46,+37:53:32.1,121.926917,37.892250,J2000,Optical,SDSS,Recoil Candidate,-100,-100,-100,-100,-100,-100,0.21,-76,Offset Broad Lines,-99,Kim+2016,2016ApJ...824..122K,https://doi.org/10.3847/0004-637X/824/2/122
3,085946.35+274534.8,0.245,17.71,0.15,0.26,11.79,-85,10,0.23,0.87,-105,5,085946.35+274534.8,085946.35+274534.8,0.245,0.245,spec,spec,08:59:46.35,+27:45:34.8,134.943125,27.759667,08:59:46.35,+27:45:34.8,134.943125,27.759667,J2000,Optical,SDSS,Recoil Candidate,-100,-100,-100,-100,-100,-100,0.23,-85,Offset Broad Lines,-99,Kim+2016,2016ApJ...824..122K,https://doi.org/10.3847/0004-637X/824/2/122
4,090654.47+391455.3,0.241,17.78,0.12,0.12,11.60,-137,36,0.37,1.40,-101,23,090654.47+391455.3,090654.47+391455.3,0.241,0.241,spec,spec,09:06:54.47,+39:14:55.3,136.726958,39.248694,09:06:54.47,+39:14:55.3,136.726958,39.248694,J2000,Optical,SDSS,Recoil Candidate,-100,-100,-100,-100,-100,-100,0.37,-137,Offset Broad Lines,-99,Kim+2016,2016ApJ...824..122K,https://doi.org/10.3847/0004-637X/824/2/122
5,091617.53+303037.9,0.215,18.40,0.10,0.17,11.48,-208,17,0.61,2.12,-236,22,091617.53+303037.9,091617.53+303037.9,0.215,0.215,spec,spec,09:16:17.53,+30:30:37.9,139.073042,30.510528,09:16:17.53,+30:30:37.9,139.073042,30.510528,J2000,Optical,SDSS,Recoil Candidate,-100,-100,-100,-100,-100,-100,0.61,-208,Offset Broad Lines,-99,Kim+2016,2016ApJ...824..122K,https://doi.org/10.3847/0004-637X/824/2/122
6,092247.03+512038.0,0.161,17.39,0.16,0.31,11.43,-212,13,0.78,2.17,-215,13,092247.03+512038.0,092247.03+512038.0,0.161,0.161,spec,spec,09:22:47.03,+51:20:38.0,140.695958,51.343889,09:22:47.03,+51:20:38.0,140.695958,51.343889,J2000,Optical,SDSS,Recoil Candidate,-100,-100,-100,-100,-100,-100,0.78,-212,Offset Broad Lines,-99,Kim+2016,2016ApJ...824..122K,https://doi.org/10.3847/0004-637X/824/2/122
7,105752.69+105037.9,0.221,17.78,K,K,K,-91,0,0.26,0.93,-137,13,105752.69+105037.9,105752.69+105037.9,0.221,0.221,spec,spec,10:57:52.69,+10:50:37.9,164.469542,10.843861,10:57:52.69,+10:50:37.9,164.469542,10.843861,J2000,Optical,SDSS,Recoil Candidate,-100,-100,-100,-100,-100,-100,0.26,-91,Offset Broad Lines,-99,Kim+2016,2016ApJ...824..122K,https://doi.org/10.3847/0004-637X/824/2/122
8,115341.16+101754.0,0.161,16.37,0.25,0.38,11.58,-202,5,0.74,2.07,-189,25,115341.16+101754.0,115341.16+101754.0,0.161,0.161,spec,spec,11:53:41.16,+10:17:54.0,178.421500,10.298333,11:53:41.16,+10:17:54.0,178.421500,10.298333,J2000,Optical,SDSS,Recoil Candidate,-100,-100,-100,-100,-100,-100,0.74,-202,Offset Broad Lines,-99,Kim+2016,2016ApJ...824..122K,https://doi.org/10.3847/0004-637X/824/2/122
9,115632.

In [ ]:
###

# HENNAWI 2013 IS PROJECTED QUASAR PAIRS WITH VELOCITY DIFFERENCES >2000 KM S^-1, NO NEED TO INCLUDE.
# Prochaska+2013 are also projected pairs with >2000 km s^-1
# Lau+2018 and Lau+2016 required velocity differences >3000 km s^-1
# Rubni 2015 expunged all systems within 5000 km s^-1 of the foreground quasar